In [12]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [13]:
pip install beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [15]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #-------------------------------------------------------------- ok
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #--------------------------------------------------------------- ok
        
    #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    

    #--------------------------------------------------------------- ok

    lista_actores_peli_i=[]
    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,12):

        #accedemos a la informacion del actor/actriz:
        try:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
            sleep(5)                    
            #-----------------------------------------------------------------ok

            #seleccionamos MENU desplegable, para acceder a biografia y premios:
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
            sleep(2)
            #acceso a la biografia
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
            sleep(2)
            #----------------------------------------------------------------ok
            
            # obtenemos el NOMBRE del actor
            nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
            sleep(5)
            #---------------------------------------------------------------- ok
            #obtenemos AÑO
            try:
                año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
                año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
            except:
                año='NULL'
            sleep(3)

            #---------------------------------------------------------------- ok

            #obtenemos el texto para filtrar informacion 'CONOCIDO POR':
            try:
                # scroll hasta el texto que necesitamos
                xpath_d='//*[@id="mini_bio"]'
                pyhton_d= driver.find_element(By.XPATH, xpath_d)
                driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
                sleep(6)
                #Obtenemos el texto:
                driver.find_element("xpath", '//*[@id="mini_bio"]').click()
                texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text
                
                sleep(6)
                conocido_por=[]
                if re.search('conocido por|conocida por', texto_actor) == None:
                        conocido_por='NULL'
                else:
                    patron_a='\.'
                    patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                    patron_c='\(\d{4}'
                    patron_d="\), |\) y |\)"
                    lista_sep=re.split(patron_b, texto_actor)
                    lista_sep_2=re.split(patron_a, lista_sep[1])
                    # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                    lista_sep_3=re.split(patron_c, lista_sep_2[0])
                        
                    for y in lista_sep_3:
                        lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                        if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                            conocido_por.append(lista_sep_3_sin_espacios)
            except:
                conocido_por='NULL'
            #---------------------------------------------------------------- ok
            sleep(2)
            driver.back() # al acceder a 'mi biografía' hacemos click. tenemos que volver atrás. SEGUIMOS DENTRO DE BIOGRAFIA ACTOR
            sleep(2)
            driver.back() # VOLVEMOS A PAGINA PRINCIPAL ACTOR.
            sleep(10)

            #--------------------------------------------------------------- ok
            #acceso a los PREMIOS, desde el desplegable.
            try:
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
            
            except:
                #si se sale de la pagina, (si no hay texto, conocido por) volvemos a ella:
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click() #acceso actor.
                sleep(2) 
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
                sleep(2)

    
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la premios.
            sleep(6)
            #creamos ciclo for para sacar las los premios:
            
            try:
                premios=[]
                for i in range(1,6): #solo queremos 5 pelis
                    premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                    premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                    premios.append(premio_limpio)
                sleep(6)
            except:
                premios='NULL'
            
            

            driver.back()
            sleep(3)

            #--------------------------------------------------------------- ok
            #extraemos informacion que hace.

            que_hace=[]
            try:
                try:
                    for z in range(1,4):
                        peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                        que_hace.append(peli_hace)
                        sleep(2)
                except:
                        peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                        que_hace.append(peli_hace)
                        sleep(2)
            except:
                que_hace='NULL'
            
            #--------------------------------------------------------------- ok

            #sacamos tupla de cada actor
            lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
            tupla_x=tuple(lista_infor_actor)
            lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula
        
            sleep(6)
            driver.back()
            #volvemos a la página principal 
        except:
            pass

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [16]:
datos_csv =pd.read_csv(r"C:\Users\Mabel\Desktop\Resto_Adalab\Proyecto_2_Interstellar\Files\sublista_selenium.csv") 

In [17]:
datos_csv


,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [18]:
id_ = datos_csv["ID_Pelicula"]

In [19]:
lista_id = id_.to_list()
print(id_)


0       tt1800301
1       tt1847719
2      tt10270648
3      tt14991492
4       tt2363363
          ...    
995    tt10106152
996    tt10363182
997     tt3857098
998    tt10095498
999     tt1067107
Name: ID_Pelicula, Length: 1000, dtype: object


In [20]:
#parte 1
lista_id_parte_1= lista_id[501:601]

In [21]:
dic_peli={'id_peli':[], 'actores_10':[]}
no_peli=[]
for id_peli in tqdm(lista_id_parte_1):
    try:
        tupla_una_peli=tuplas_actores(id_peli)
        print(tupla_una_peli)
        dic_peli['actores_10'].append(tupla_una_peli)
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        print('fallo peli:--------->', id_peli)
        continue

    
print(dic_peli)

  0%|          | 0/100 [00:00<?, ?it/s]

fallo peli:---------> tt11756362


  2%|▏         | 2/100 [12:37<12:05:58, 444.48s/it]

[('Jack Devnarain', '1971', ['Reparto', 'Reparto'], ['Los demonios de la noche', 'Isidingo', '31 Million Reasons'], 'NULL'), ('Luthuli Dlamini', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en The Furnace', 'Blood Drive', "Mandela's Gun"], 'NULL'), ('Bheki Nkosi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL'), ('Thabisile Zikhali', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway', 'Wake Up Sade'], 'NULL'), ('Renos Spanoudes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway', 'The Drop', 'Taryn & Sharon'], 'NULL'), ('Simphiwe Kunene', 'NULL', ['Especialistas', 'Reparto', 'Sonido'], ["su papel en Big Nunu's Little Heist", 'Intlawulo', 'Alleyway'], 'NULL'), ('Linda Masina', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL'), ('Bheki Khoza', 'NULL', ['Reparto', 'Música', 'Reparto'], ['su papel en Alleyway', 'Sophiatown', 'Shaka iLembe'], 'NULL'), ('Righteous Mongwe', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL

  3%|▎         | 3/100 [12:43<6:34:42, 244.15s/it] 

...

[('Toni Luoma-Nirva', 'NULL', ['Edición', 'Edición', 'Reparto'], ['su papel en Fight or Flight', 'The Pain of Creation', 'Toni Air'], 'NULL'), ('Anton Jokikunnas', 'NULL', ['Cámara y electricidad', 'Dirección', 'Producción'], ['su papel en Angels of the Dark MC, Layehaye Doroogh', 'Viraali'], 'NULL'), ('Juha Mäkinen', 'NULL', ['Reparto', 'Edición', 'Dirección'], ['su papel en Live'], 'NULL')]
fallo peli:---------> tt0493949


 98%|█████████▊| 98/100 [7:56:14<06:34, 197.32s/it]

[('Judy Krause', 'NULL', ['Reparto', 'Producción', 'Dirección de fotografía y cámara'], ['su papel en Brooklyn in July', 'A Rogue in Londinium', "Adams' Apples"], 'NULL'), ('Travis Mitchell', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Spinning Man', 'Partitioned Heart', 'Nevertheless, She Auditioned'], 'NULL'), ('Bob Rutan', 'NULL', ['Equipos adicionales', 'Reparto', 'Producción'], ['su papel en Beached', 'Ten Dollars', "Macy's Fourth of July Fireworks Spectacular"], 'NULL')]
fallo peli:---------> tt2376726


100%|██████████| 100/100 [8:09:06<00:00, 293.46s/it]

[('Tovino Thomas', '1989', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], ['Rayo Murali', '2018', 'Kala'], ['Asianet Film Awards 2019 Ganador Asianet Film Award Performer of the Year 2018 Ganador Asianet Film Award Performer of the Year Mayaanadhi', 'Filmfare Awards South 2022 Nominado Filmfare Award - Malayalam Film Industry Best Actor Rayo Murali 2019 Nominado Filmfare Award - Malayalam Film Industry Best Actor Theevandi 2018 Nominado Filmfare Award - Malayalam Film Industry Best Actor Mayaanadhi', 'South Indian International Movie Awards 2023 Ganador SIIMA - Malayalam Best Actor in a Leading Role Thallumaala 2022 Ganador SIIMA - Malayalam Best Actor in a Leading Role Rayo MuraliKala 2021 Nominado SIIMA - Malayalam Best Actor in a Leading Role Forensic 2019 Ganador SIIMA - Malayalam Best Actor in a Leading Role Theevandi', 'Asiavision Awards 2015 Ganador Asiavision Film Award Best Actor - Supporting Role Ennu Ninte Moideen', 'Vanitha Film Awards

In [1]:
dic_peli={'id_peli': ['tt13210534', 'tt10763820', 'tt23803320', 'tt13886150', 'tt1666538', 'tt11004574', 'tt2301155', 'tt31806125', 'tt2849536', 'tt30755280', 'tt14724692', 'tt26448876', 'tt10198898', 'tt12261776', 'tt13989030', 'tt0961036', 'tt10265126', 'tt10192466', 'tt28341583', 'tt11434772', 'tt11567240', 'tt30972810', 'tt3317570', 'tt11853708', 'tt11666734', 'tt12991676', 'tt10758124', 'tt12408808', 'tt1640130', 'tt1742096', 'tt13882736', 'tt10284690', 'tt10548482', 'tt12879494', 'tt30291879', 'tt10007932', 'tt10507176', 'tt1604577', 'tt0787442', 'tt23330510', 'tt14506528', 'tt10212486', 'tt12631784', 'tt2441174', 'tt28677683', 'tt2674046', 'tt1196124', 'tt10280636', 'tt10039088', 'tt10027194', 'tt10026214', 'tt10687220', 'tt10613292', 'tt10736172', 'tt12632230', 'tt10836830', 'tt23469336', 'tt2539454', 'tt10488264', 'tt24220088', 'tt10218838'], 'actores_10': [[('Jack Devnarain', '1971', ['Reparto', 'Reparto'], ['Los demonios de la noche', 'Isidingo', '31 Million Reasons'], 'NULL'), ('Luthuli Dlamini', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en The Furnace', 'Blood Drive', "Mandela's Gun"], 'NULL'), ('Bheki Nkosi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL'), ('Thabisile Zikhali', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway', 'Wake Up Sade'], 'NULL'), ('Renos Spanoudes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway', 'The Drop', 'Taryn & Sharon'], 'NULL'), ('Simphiwe Kunene', 'NULL', ['Especialistas', 'Reparto', 'Sonido'], ["su papel en Big Nunu's Little Heist", 'Intlawulo', 'Alleyway'], 'NULL'), ('Linda Masina', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL'), ('Bheki Khoza', 'NULL', ['Reparto', 'Música', 'Reparto'], ['su papel en Alleyway', 'Sophiatown', 'Shaka iLembe'], 'NULL'), ('Righteous Mongwe', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL'), ('Akim Andile Nkabinde', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alleyway'], 'NULL')], [('Jorge Lendeborg Jr.', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], ['su papel en Bumblebee', 'Spider-Man: Homecoming', 'Fauces de la noche'], 'NULL'), ('Debby Ryan', '1993', ['Reparto', 'Música', 'Producción'], ['Insatiable', 'Sing It!', 'Jessie'], ["Kids' Choice Awards, USA 2016 Nominado Blimp Award Favorite TV Actress Jessie 2015 Nominado Blimp Award Favorite TV Actress Jessie 2014 Nominado Blimp Award Favorite TV Actress Jessie", 'Teen Choice Awards 2014 Nominado Teen Choice Award Choice TV Actress: Comedy Jessie', 'Young Artist Awards 2011 Nominado Young Artist Award Best Performance in a TV Movie, Miniseries or Special - Leading Young Actress 16 deseos', 'Behind the Voice Actors Awards 2013 Nominado BTVA Special/DVD Voice Acting Award Best Vocal Ensemble in a TV Special/Direct-to-DVD Title or Short Campanilla: El secreto de las hadas Compartido con: Mae Whitman · Lucy Hale · Timothy Dalton · Jeff Bennett · Lucy Liu · Raven-Symoné · Megan Hilty · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Lucy Fry', '1992', ['Reparto', 'Reparto'], ['Bright', '22'], 'NULL'), ('Raúl Castillo', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['Cassandro', 'We the Animals', 'Ejército de los muertos'], ['Denver International Film Festival 2022 Ganador Excellence in Acting Award', 'Gotham Awards 2022 Nominado Gotham Independent Film Award Outstanding Supporting Performance La inspección', 'Imagen Foundation Awards 2015 Nominado Imagen Award Best Supporting Actor - Television Looking 2014 Nominado Imagen Award Best Actor/Television Looking', 'Film Independent Spirit Awards 2019 Nominado Independent Spirit Award Best Supporting Male We the Animals', 'NAMIC Vision Awards 2016 Nominado Vision Award Best Performance - Comedy Looking 2015 Ganador Vision Award Best Performance - Comedy Looking Tied with Don Cheadle for House of Lies.']), ('Alfie Allen', '1986', ['Reparto', 'Banda sonora', 'Reparto'], ['John Wick (Otro día para matar', 'Juego de tronos', 'Predator'], ['Primetime Emmy Awards 2019 Nominado Primetime Emmy Outstanding Supporting Actor in a Drama Series Juego de tronos For playing: "Theon Greyjoy".', 'Royal Television Society, UK 2018 Nominado RTS Southern Television Award Best Factual Series Football: A Brief History by Alfie Allen Compartido con: Mike Christie', 'Screen Actors Guild Awards 2020 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Juego de tronos Compartido con: Pilou Asbæk · Jacob Anderson · John Bradley · Gwendoline Christie · Emilia Clarke · Nikolaj Coster-Waldau · · · · · · · · · · · · · · · · · · · · · · 2020 Nominado Actor Outstanding Performance by a Cast in a Motion Picture Jojo Rabbit Compartido con: Roman Griffin Davis · Scarlett Johansson · Thomasin McKenzie · Stephen Merchant · Sam Rockwell · · 2018 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Juego de tronos Compartido con: Jacob Anderson · Pilou Asbæk · Hafþór Júlíus Björnsson · John Bradley · Jim Broadbent · Gwendoline Christie · · · · · · · · · · · · · · · · · · · · · · 2017 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Juego de tronos Compartido con: Jacob Anderson · Dean-Charles Chapman · Emilia Clarke · Nikolaj Coster-Waldau · Liam Cunningham · · · · · · · · · · · · · 2016 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Juego de tronos Compartido con: Ian Beattie · John Bradley · Gwendoline Christie · Emilia Clarke · Michael Condron · Nikolaj Coster-Waldau · · · · · · · · · · · · · · · · · · · · · · · 2 más', 'Savannah Film Festival 2015 Ganador Rising Star Award', "IGN Summer Movie Awards 2019 Nominado IGN Award Best TV Ensemble Juego de tronos Compartido con: Peter Dinklage · Lena Headey · Emilia Clarke · Kit Harington · Sophie Turner · Maisie Williams · · · · · · · · · · · · · 2019 Ganador IGN People's Choice Award Best TV Ensemble Juego de tronos Compartido con: Peter Dinklage · Lena Headey · Emilia Clarke · Kit Harington · Sophie Turner · Maisie Williams · · · · · · · · · · · · ·"]), ('Marlene Forte', '1961', ['Reparto', 'Producción', 'Dirección'], ['Star Trek', 'Paranormal Movie', 'Puñales por la espalda (Knives Out'], ['Imagen Foundation Awards 2009 Nominado Imagen Award Best Supporting Actress/Television La desaparición de Delimar', 'CinEuphoria Awards 2024 Ganador CinEuphoria Merit - Honorary Award Fear the Walking Dead Compartido con: Michael E. Satrazemis · Austin Amelio · Rubén Blades · Dayton Callie · Keith Carradine · Zoe Colletti · · · · · · · · · · · · · · · · · · · · · · · ·', 'Online Film & Television Association 2020 Nominado OFTA Film Award Best Ensemble Puñales por la espalda (Knives Out) Compartido con: Daniel Craig · Chris Evans · Ana de Armas · Jamie Lee Curtis · Michael Shannon · Don Johnson · · · · · · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Ash Santos', '1993', ['Reparto', 'Reparto'], ['Mayor of Kingstown', 'Mike', 'Una historia muy real'], 'NULL'), ('Nandy Martin', 'NULL', ['Reparto', 'Reparto'], ['su papel en El mentalista', 'Fauces de la noche', 'Cuentos asombrosos'], 'NULL'), ('Jaren Mitchell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Queen Sugar', 'The Purge', 'Los tortolitos'], 'NULL'), ('Robert Larriviere', 'NULL', ['Reparto', 'Reparto'], ['su papel en La chica salvaje', 'The Girl from Plainville', '2 Guns'], 'NULL')], [('Charlotte Clarke', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Cámara y electricidad'], ['su papel en Homozygous', 'Little Shaw Lane', 'Spellbound'], 'NULL'), ('Taylor Jayne Claxton', '1999', ['Dirección', 'Reparto', 'Guion'], ['Unrequited', 'Edge', 'Spellbound'], 'NULL'), ('William Dell-Price', 'NULL', ['Reparto', 'Reparto'], ['su papel en Foils y Spellbound'], 'NULL'), ('Patrícia Inverneiro', 'NULL', ['Reparto', 'Reparto'], ['su papel en Finding Peace', 'The Bad Taste Show', 'Bitter Taste'], 'NULL'), ('Maria Lisberg-Jonasson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Escaping the Dead', 'Spellbound'], 'NULL')], [], [('Jonathan Baldock', 'NULL', ['Reparto', 'Producción', 'Especialistas'], ['su papel en A Case of Deceit', 'Zombie Killers', 'Conduct Unbecoming'], 'NULL'), ('Leslye-Aikayla So', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit'], 'NULL'), ('Viktor Radics', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit'], 'NULL'), ('David Kinsman', 'NULL', ['Reparto', 'Reparto'], ['su papel en Call of Duty: Find Makarov', 'A Case of Deceit', 'Forbidden Science'], 'NULL'), ('J.D. Coulter', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en A Case of Deceit', 'Audition', 'Death Fuck 2'], 'NULL'), ('Michael Hannigan', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit', 'Lucky 7'], 'NULL'), ('Julian Nicholson', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit', 'Dark Fist', 'Lucky 7'], 'NULL'), ('Monty Greig', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit', 'Mail Order Bride'], 'NULL'), ("Walter D'Amico", 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit'], 'NULL'), ('Bruno Roder', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Case of Deceit', 'A Cry in the Dark', 'Logarithm'], 'NULL')], [('Alicia Hechavarría', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ciudad en rojo', 'Fabula', 'The Foam of Days'], 'NULL'), ('Jorge Luis Álvarez', 'NULL', ['Reparto', 'Dirección de fotografía y cámara', 'Edición'], ['su papel en Queen of Thursdays', 'Papeles secundarios', 'Una novia para David'], 'NULL'), ('Fernando Hechavarria', 'NULL', ['Reparto', 'Reparto'], ['su papel en La tierra y el cielo', 'Regreso a Ítaca', 'The Foam of Days'], 'NULL'), ('Carlos Miguel Caballero', 'NULL', ['Reparto', 'Reparto'], ['su papel en María de los Ángeles', 'Prisionera', 'Periodistas'], 'NULL'), ('Yaniel Castillo', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Latin Boys: Volume 1', 'Mountains', 'Mila Caos'], 'NULL'), ('Matthew Ryan', 'NULL', ['Reparto', 'Reparto'], ['su papel en El Dragón: El regreso de un guerrero', 'A Place in the Dark y Shiny New Things'], 'NULL')], [('Anil Kapoor', '1956', ['Reparto', 'Producción', 'Música'], ['Slumdog Millionaire', 'Animal', 'Race 3'], ['Filmfare Awards 2024 Nominado Filmfare Award Best Supporting Actor Animal 2023 Ganador Filmfare Award Best Supporting Actor Jug Jugg Jeeyo 2016 Ganador Filmfare Award Best Supporting Actor Dil Dhadakne Do 2008 Nominado Filmfare Award Best Supporting Actor Welcome 2006 Nominado Filmfare Award Best Comic Actor No Entry 14 más', 'Awards of the International Indian Film Academy 2023 Ganador IIFA Award Best Actor in a Supporting Role - Male Jug Jugg Jeeyo 2019 Nominado Popular Award Best Actor in a Supporting Role Race 3 2016 Ganador Popular Award Best Actor in a Supporting Role Dil Dhadakne Do 2011 Nominado Popular Award Best Comic Actor No Problem 2010 Ganador Special Award Outstanding Achievement in Cinema Compartido con: Rakeysh Omprakash Mehra · Zeenat Aman 6 más', "Zee Cine Awards 2024 Nominado Jury's Choice Award Best Supporting Actor Animal 2023 Ganador Jury's Choice Award Best Supporting Actor Jug Jugg Jeeyo 2018 Ganador Jury's Choice Award Best Actor in a Supporting Role Mubarakan 2016 Ganador Popular Award International Icon - Male 2016 Nominado Critics Choice Award Best Supporting Actor Dil Dhadakne Do 6 más", 'National Film Awards, India 2001 Ganador Silver Lotus Award Best Actor Pukar', 'Screen Actors Guild Awards 2009 Ganador Actor Outstanding Performance by a Cast in a Motion Picture Slumdog Millionaire Compartido con: Rubina Ali · Tanay Chheda · Ashutosh Lobo Gajiwala · Azharuddin Mohammed Ismail · Irrfan Khan · · · · ·']), ('John Abraham', '1972', ['Reparto', 'Producción', 'Música'], ['Dhoom', 'Zinda', 'Vicky Donor'], ['Filmfare Awards 2013 Nominado Filmfare Award Best Film Vicky Donor Compartido con: Sunil Lulla · Ronnie Lahiri · Ram Mirchandani 2007 Nominado Filmfare Award Best Supporting Actor Baabul 2007 Nominado Filmfare Award Best Performance by an Actor in a Negative Role Zinda 2005 Nominado Filmfare Award Best Performance by an Actor in a Villainous Role Dhoom', 'Awards of the International Indian Film Academy 2013 Nominado Popular Award Best Picture Vicky Donor Compartido con: Ronnie Lahiri 2012 Nominado Special Award Star Plus Hottest Pair Desi Boyz Compartido con: Akshay Kumar 2007 Nominado Popular Award Best Performance by an Actor in a Negative Role Zinda 2006 Nominado Popular Award Best Actor in a Supporting Role Garam Masala 2005 Ganador Popular Award Best Performance by an Actor in a Negative Role Dhoom', "Zee Cine Awards 2024 Nominado Jury's Choice Award Best Performance by an Actor in a Negative Role Pathaan 2012 Nominado Special Award Male International Icon 2007 Nominado Popular Award Best Performance By an Actor in a Negative Role Zinda 2005 Ganador Popular Award Best Performance by an Actor in a Villainous Role Dhoom", 'National Film Awards, India 2013 Ganador Golden Lotus Award Best Popular Film Providing Wholesome Entertainment Vicky Donor Compartido con: Shoojit Sircar', "Bollywood Awards 2007 Nominado Bollywood Movie Award Best Supporting Actor Baabul 2005 Nominado Bollywood Movie Award Best Actor in a Villain's Role Dhoom 2004 Ganador Bollywood Movie Award Best Debutant - Male El lado oscuro del deseo"]), ('Manoj Bajpayee', '1969', ['Reparto', 'Producción', 'Música'], ['Gangs of Wasseypur', 'Satya', 'Aks'], ["Filmfare Awards 2024 Nominado Critics' Awards Best Actor Joram 2017 Ganador Critics' Awards Best Actor Aligarh Tied with Shahid Kapoor for Udta Punjab (2016) 2017 Ganador Short Film Award Best Actor in a Short Film Taandav 2013 Nominado Filmfare Award Best Actor Gangs of Wasseypur 2011 Nominado Filmfare Award Best Supporting Actor Raajneeti 7 más", 'Awards of the International Indian Film Academy 2022 Nominado IIFA Award Best Actor in a Leading Role - Male Bhonsle 2013 Nominado Popular Award Best Actor in a Leading Role Gangs of Wasseypur 2011 Nominado Popular Award Best Actor in a Supporting Role Raajneeti 2011 Nominado Popular Award Best Performance by an Actor in a Negative Role Raajneeti 2002 Nominado Popular Award Best Performance by an Actor in a Negative Role Aks 1 más', "Zee Cine Awards 2024 Ganador Jury's Choice Award Best Actor - OTT Sirf Ek Bandaa Kaafi Hai 2017 Nominado Critics Choice Award Best Actor Aligarh 2012 Nominado Critics Choice Award Best Villain Aarakshan 2011 Nominado Popular Award Best Performance by an Actor in a Negative Role Raajneeti 2003 Nominado Popular Award Best Performance By an Actor in a Negative Role Road 3 más", 'National Film Awards, India 2020 Ganador Silver Lotus Award Best Actor Bhonsle 2004 Ganador Special Jury Award Special Jury Award Pinjar: Beyond Boundaries... For his portrayal of the dilemma of a man caught between the dictates of his community and his love for his wife. 1999 Ganador Silver Lotus Award Best Supporting Actor Satya', 'New York Indo-American Arts Council Film Festival 2018 Ganador Festival Award Best Actor Gali Guleiyan']), ('Tusshar Kapoor', '1976', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], ['Golmaal: Fun Unlimited', 'Golmaal Returns', 'Golmaal 3'], ['Filmfare Awards 2009 Nominado Filmfare Award Best Supporting Actor Golmaal Returns 2007 Nominado Filmfare Award Best Comic Actor Golmaal: Fun Unlimited', 'Awards of the International Indian Film Academy 2009 Nominado Popular Award Best Performance in a Comic Role Golmaal Returns 2007 Ganador Popular Award Best Comic Actor Golmaal: Fun Unlimited', 'Zee Cine Awards 2008 Nominado Popular Award Best Comic Actor Dhol 2007 Nominado Popular Award Best Comic Actor Golmaal: Fun Unlimited 2002 Ganador Popular Award Best Debut - Male Mujhe Kucch Kehna Hai', 'Bollywood Awards 2007 Nominado Bollywood Movie Award Best Comic Actor Golmaal: Fun Unlimited 2002 Ganador Bollywood Movie Award Best Debut Actor Mujhe Kucch Kehna Hai', "People's Choice Awards, India 2012 Nominado Movie Award Favourite Ensemble Cast The Dirty Picture Compartido con: Vidya Balan · Emraan Hashmi · Naseeruddin Shah · Rajesh Sharma · Mangala Kenkre · Anju Mahendru · 2001 Nominado People's Choice Award Best Male Debut Mujhe Kucch Kehna Hai"]), ('Kangana Ranaut', '1987', ['Reparto', 'Dirección', 'Producción'], ['Queen', 'Tanu Weds Manu Returns', 'Tanu Weds Manu'], ["Filmfare Awards 2021 Nominado Filmfare Award Best Actress Panga 2020 Nominado Filmfare Award Best Actress Manikarnika 2020 Nominado Critics' Awards Best Actress Judgementall Hai Kya 2018 Nominado Critics' Awards Best Actress Rangoon 2016 Ganador Critics' Awards Best Actress Tanu Weds Manu Returns 5 más", 'Awards of the International Indian Film Academy 2016 Nominado Popular Award Best Actress in a Leading Role Tanu Weds Manu Returns 2015 Ganador Popular Award Best Actress in a Leading Role Queen 2014 Nominado Popular Award Best Actress in a Supporting Role Krrish 3 2012 Nominado Popular Award Best Actress in a Leading Role Tanu Weds Manu 2009 Ganador Popular Award Best Actress in a Supporting Role Fashion 2 más', "Zee Cine Awards 2020 Nominado Jury's Choice Award Best Actress Manikarnika 2016 Nominado Popular Award Best Actress Tanu Weds Manu Returns 2016 Nominado Critics Choice Award Best Actress Tanu Weds Manu Returns 2014 Nominado Popular Award Best Actress in a Supporting Role Krrish 3 2012 Nominado Popular Award Best Actress Tanu Weds Manu 2 más", 'National Film Awards, India 2020 Ganador Silver Lotus Award Best Actress ManikarnikaPanga 2016 Ganador Silver Lotus Award Best Actress Tanu Weds Manu Returns For her dynamic portrayal of two contrast characters, laced with mirth and charm. 2015 Ganador Silver Lotus Award Best Actress Queen For an endearingly etched out performance that is grounded in contextual specificity and at the same time is disarmingly spontaneous as to appear almost improvisatory. 2009 Ganador Silver Lotus Award Best Supporting Actress Fashion For compelling portrayal of a down and out super model that enriches the impact of the film.', 'Apsara Film Producers Guild Awards 2016 Nominado Apsara Award Best Actress Tanu Weds Manu Returns 2015 Nominado Apsara Award Best Actress - Movie Queen 2015 Nominado Apsara Award Dialogue of the year Queen Main india se hoon, Rajouri. Rajouri suna hai? Apne honeymoon pe akele aayi hoon! Compartido con: Anvita Dutt 2014 Nominado Apsara Award Best Actress In A Supporting Role Krrish 3 2012 Nominado Apsara Award Best Actress - Movie Tanu Weds Manu 1 más']), ('Sonu Sood', '1973', ['Reparto', 'Producción', 'Dirección'], ['Dabangg', 'Jodhaa Akbar', 'Dookudu'], ['Filmfare Awards 2009 Nominado Filmfare Award Best Supporting Actor Jodhaa Akbar', 'Awards of the International Indian Film Academy 2011 Ganador Popular Award Best Performance by an Actor in a Negative Role Dabangg 2009 Nominado Popular Award Best Actor in a Supporting Role Jodhaa Akbar', "Zee Cine Awards 2019 Nominado Jury's Choice Award Best Performance by an Actor Negative Role Simmba 2011 Nominado Popular Award Best Performance by an Actor in a Negative Role Dabangg", 'Apsara Film Producers Guild Awards 2011 Ganador Apsara Award Best Performance by an Actor in a Negative Role Dabangg 2009 Nominado Apsara Award Best Actor In A Supporting Role Jodhaa Akbar', 'Filmfare Awards South 2012 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Kandireega 2010 Ganador Filmfare Award - Telugu Film Industry Best Supporting Actor Arundhati']), ('Ronit Roy', '1965', ['Reparto', 'Equipos adicionales', 'Dirección de la segunda unidad o asistencia a dirección'], ['2 States', 'Kaabil', 'Udaan'], ['Filmfare Awards 2015 Nominado Filmfare Award Best Supporting Actor 2 States 2011 Ganador Filmfare Award Best Supporting Actor Udaan', 'Awards of the International Indian Film Academy 2015 Nominado Popular Award Best Actor in a Supporting Role 2 States 2011 Nominado Popular Award Best Performance by an Actor in a Negative Role Udaan', "Zee Cine Awards 2018 Nominado Jury's Choice Award Best Negative Role Kaabil 2011 Ganador Popular Award Best performance by an Actor in a Negative Role Udaan", 'Apsara Film Producers Guild Awards 2015 Ganador Apsara Award Best Actor - TV Itna Karo Na Mujhe Pyaar Shared the award with Mohit Raina 2015 Nominado Apsara Award Best Actor in a Supporting Role - Movie 2 States 2011 Nominado Apsara Award Best Performance by an Actor in a Negative Role Udaan 2011 Nominado Apsara Award Best Actor - TV Bandini 2006 Nominado Apsara Award Best Actor - TV Kyunki Saas Bhi Kabhi Bahu Thi 1 más', 'Indian Television Academy Awards, India 2016 Nominado Indian Television Academy Award Best Actor - Drama Adaalat (Season 2) 2009 Ganador Indian Television Academy Award Best Actor - Drama Bandini 2004 Ganador Indian Television Academy Award Best Actor - Popular Kyunki Saas Bhi Kabhi Bahu Thi 2003 Ganador Indian Television Academy Award Best Actor - Popular Kyunki Saas Bhi Kabhi Bahu Thi']), ('Mahesh Manjrekar', '1953', ['Guion', 'Producción', 'Reparto'], ['Kaante', 'Slumdog Millionaire', 'Astitva'], ['Filmfare Awards 2003 Nominado Filmfare Award Best Comic Actor Kaante 2000 Nominado Filmfare Award Best Director Vaastav: The Reality', 'Awards of the International Indian Film Academy 2000 Nominado Popular Award Best Director Vaastav: The Reality', 'Zee Cine Awards 2003 Nominado Popular Award Best Actor in a Supporting Role Kaante', 'National Film Awards, India 2001 Ganador Silver Lotus Award Best Regional Film (Marathi) Astitva Compartido con: Jhamu Sughand', 'Bollywood Awards 2003 Nominado Bollywood Movie Award Best Comic Actor Kaante 2003 Nominado Bollywood Movie Award Best Supporting Actor Kaante']), ('Raju Mavani', '1957', ['Reparto', 'Producción', 'Dirección'], ['Surakshaa', 'Ram Aur Shyam', 'Iski Topi Uske Sarr'], 'NULL'), ('Arif Zakaria', '1966', ['Reparto', 'Producción', 'Reparto'], ['Raazi', 'Mi nombre es Khan', 'War'], 'NULL')], [('Wang Wang', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Never', 'Until the End of Time', 'Missing is a knife'], ['Accolade Competition 2024 Ganador Award of Excellence Film Short Never 2024 Ganador Award of Excellence Editing Never 2024 Ganador Award of Merit Special Mention Art Direction Never', 'Best Shorts Competition 2023 Ganador Award of Merit Film Short Never 2023 Ganador Award of Merit Leading Actor Never', 'IndieFEST Film Awards 2023 Ganador Award of Excellence Film Short Never 2023 Ganador Award of Merit Special Mention Direction Never', 'International Open Film Festival (IOFF) 2024 Ganador Best Award Best Short Film Never', 'Cinema World Fest Awards 2024 Ganador Award of Recognition Best Drama, Short Never 2024 Ganador Award of Merit Best Actor Never']), ('Xiang Li', 'NULL', ['Reparto', 'Reparto'], ['su papel en Never', 'Missing is a knife'], 'NULL')], [('Nick Liam Heaney', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Zero Day, Urgencias', 'Dead Drop'], 'NULL'), ('Brian McNamara', '1960', ['Reparto', 'Producción', 'Dirección'], ['Army Wives', 'Navy: Investigación criminal', 'Aracnofobia'], 'NULL'), ('Basil Hoffman', '1938', ['Reparto', 'Equipos adicionales', 'Reparto'], ['The box', '¡Ave, César!', 'The Artist'], ['Couch Film Festival 2022 Ganador Best Actor Best Actor Third Act', 'Hollywood International Moving Pictures Film Festival 2019 Ganador HIMPFF Award Best Actor Mr. Roberts', 'Bloodstained Indie Film Festival 2019 Ganador Festival Award Best Actor Mr. Roberts', 'Beverly Hills Shorts Festival 2009 Ganador Best Actor Best Actor Old Dogs', 'Global Film Festival Awards 2022 Ganador Global Film Festival Award Best Ensemble Cast Third Act Compartido con: Jodi Lynn Thomas · Kelsey Leos Montoya · Kristin K. Berg · Vic Browder · Keith Allen West · Doug Montoya · · ·']), ('John Cothran', '1947', ['Reparto', 'Producción', 'Reparto'], ['Di que sí', 'Black Snake Moan', 'Rango'], 'NULL'), ('Konstantin Lavysh', 'NULL', ['Reparto', 'Equipos adicionales', 'Producción'], ['su papel en Power Book IV: Force', 'Call of Duty: Modern Warfare', 'The Americans'], 'NULL'), ('Jerry Marr', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en A Better Life, Once in a Blue Moon y Fragments'], 'NULL'), ('Jessica Hayes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dead Drop', 'A-Holes Anonymous', 'Hey Girl!'], 'NULL'), ('Alexandria Provenghi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Zulu Six: Revelations', 'Navy SEALS v Demons', 'Dead Drop'], 'NULL'), ('Leann Van Mol', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Downside of Bliss', 'Smuggling in Suburbia', 'After Lately'], 'NULL'), ('Ben Whitehair', 'NULL', ['Reparto', 'Equipo ejecutivo', 'Reparto'], 'NULL', 'NULL')], [('Stanescu Motatu Demetrio Leonidas', '1974', ['Dirección', 'Reparto', 'Guion'], ['Un-Mirrored', 'Dawn of Spectral Darkness, Ghaar', 'Dawn of Spectral Darkness'], ['Alternative Film Festival 2021 Ganador Spring Award Quarantine Film Dawn of Spectral Darkness', 'Lift-Off Global Network, London 2020 Nominado Best Short Film International Short Live Action Narratives Memories of SorrowSupernova 2020 Nominado Best Short Film International Coffee with my friend Jojo 2020 Nominado Best Short Film International Kim Eng Chicken Rice 2020 Nominado Best Short Film International Jojo in South of Spain 2020 Nominado Best Short Film International Under Deep Waters 4 más', 'L.A. Neo Noir Novel, Film & Script Festival 2019 Nominado Honorable Mention Best Neo Noir Actor Dark Dawn in Tokyo 2019 Ganador Most Iconic Neo Noir Actor Best Neo Noir Actor Memories of Sorrow', 'Top Shorts Film Festival 2019 Nominado September Award Best Supporting Actor Memories of Sorrow', 'Diverse Voices 2022 Ganador Festival Prize Quarter Finalist Local-ized'])], [('Catherine Asapokhai-Utsalo', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Tikima Bowers', 'NULL', ['Reparto', 'Reparto'], ['su papel en Justice Through Redemption', 'The Black Revenger', 'Eighteen'], 'NULL'), ('Harriet Burnette', 'NULL', ['Reparto', 'Dirección', 'Vestuario'], ['su papel en My 3 Boys', "Life's Cycles", 'Busted & Disgusted'], 'NULL'), ('Rachel Cain', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Jazmine Caldwell', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Kiani Conway', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Quanita Conway', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Brenda Cooper-Hardy', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL'), ('Victoria Crosby', 'NULL', ['Reparto', 'Reparto'], ['su papel en 17 Days', "Life's Cycles"], 'NULL'), ('Malik Davis', 'NULL', ['Reparto', 'Reparto'], ["su papel en Life's Cycles"], 'NULL')], [('Chris Nathan Dylan', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en The road of glory', "Shazam! The Battle of the God's y At the edge of today"], 'NULL')], [('John Yancey', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen', 'Chasing Rain', 'Spite'], 'NULL'), ('Nolan Harper', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen', 'Just Jaxzz y Just Once'], 'NULL'), ('Matt Pettigrew', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen', 'Alone Inside', 'Degradation'], 'NULL'), ('Nana Agyeman', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen'], 'NULL'), ('Averly Belle Burns', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen', 'Still The One'], 'NULL'), ('Benton Cockerell', 'NULL', ['Reparto', 'Dirección', 'Guion'], 'NULL', 'NULL'), ('Amy Elizabeth Massey', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fallen', 'Three Dates in Texas'], 'NULL'), ('Noah Scibana', 'NULL', ['Reparto', 'Reparto'], 'NULL', 'NULL')], [('Adam Driver', '1983', ['Reparto', 'Producción', 'Banda sonora'], ['Star Wars: El despertar de la Fuerza', 'Paterson', 'Star Wars: Los últimos jedi'], ['Academy Awards, USA 2020 Nominado Oscar Best Performance by an Actor in a Leading Role Historia de un matrimonio 2019 Nominado Oscar Best Performance by an Actor in a Supporting Role Infiltrado en el KKKlan', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2021 Nominado Saturn Award Best Supporting Actor Star Wars: El ascenso de Skywalker 2016 Ganador Saturn Award Best Supporting Actor Star Wars: El despertar de la Fuerza', 'American Comedy Awards, USA 2014 Nominado American Comedy Award Funniest Supporting Male Performer in a TV Series Girls 2014 Nominado American Comedy Award Best Comedy Supporting Actor - Film Amigos de más 2012 Nominado American Comedy Award Breakout Performer of the Year', 'Film Critics Circle of Australia Awards 2015 Nominado FCCA Award Best Actor - Supporting Role El viaje de tu vida', 'BAFTA Awards 2020 Nominado BAFTA Film Award Best Leading Actor Historia de un matrimonio 2019 Nominado BAFTA Film Award Best Supporting Actor Infiltrado en el KKKlan']), ('Ariana Greenblatt', 'NULL', ['Reparto', 'Reparto'], ['Barbie', 'Ahsoka', '65'], ['Critics Choice Awards 2024 Nominado Critics Choice Award Best Young Actor/Actress Barbie', 'Las Vegas Film Critics Society Awards 2023 Nominado Sierra Award Youth in Film - Female Barbie', 'Screen Actors Guild Awards 2024 Nominado Actor Outstanding Performance by a Cast in a Motion Picture Barbie Compartido con: Michael Cera · Will Ferrell · America Ferrera · Ryan Gosling · Kate McKinnon · Helen Mirren · Rhea Perlman · ·', 'Washington DC Area Film Critics Association Awards 2023 Nominado WAFCA Award Best Youth Performance Barbie', 'Hollywood Music In Media Awards (HMMA) 2021 Nominado Hollywood Music In Media Awards (HMMA) Best Original Song - Animated Film El bebé jefazo: Negocios de familia "Together We Stand" Compartido con: Gary Barlow']), ('Chloe Coleman', '2008', ['Reparto', 'Banda sonora', 'Reparto'], ['Dungeons & Dragons: Honor entre ladrones', 'Juego de espías', 'Big Little Lies'], 'NULL'), ('Nika King', 'NULL', ['Reparto', 'Producción', 'Guion'], ['Euphoria', 'Greenleaf', 'Amigas a destiempo'], 'NULL'), ('Brian Dare', 'NULL', ['Reparto', 'Reparto'], ['su papel en Jane the Virgin', 'Feud', 'Pequeñas mentirosas'], 'NULL')], [('Laura Marano', '1995', ['Reparto', 'Música', 'Producción'], ['Tratamiento real', 'La cita perfecta', 'Saving Zoë'], ["Kids' Choice Awards, USA 2016 Nominado Blimp Award Favorite TV Actress Austin y Ally 2015 Ganador Blimp Award Favorite TV Actress Austin y Ally", 'Teen Choice Awards 2019 Ganador Teen Choice Award Choice Comedy Movie Actress La cita perfecta 2019 Nominado Teen Choice Award Choice Ship La cita perfecta Compartido con: Noah Centineo 2016 Nominado Teen Choice Award Choice TV Actress: Comedy Austin y Ally 2015 Nominado Teen Choice Award Choice TV: Chemistry Austin y Ally Compartido con: Ross Lynch 2015 Nominado Teen Choice Award Choice Summer TV Star: Female Austin y Ally 1 más', 'WorldFest Houston 2015 Ganador Gold Remi Award Rising Star - Actress A Sort of Homecoming', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Mena Massoud', '1991', ['Reparto', 'Producción', 'Guion'], ['Aladdín', 'Tratamiento real', 'The Sacrifice Game'], ['Teen Choice Awards 2019 Nominado Teen Choice Award Choice Sci-Fi/Fantasy Movie Actor Aladdín', 'Canadian Screen Awards, CA 2024 Nominado Canadian Screen Award Best Host, Lifestyle Evolving Vegan', 'National Film and Television Awards, USA 2019 Nominado National Film and Television Award Best Actor Aladdín 2019 Nominado National Film and Television Award Best Newcomer', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Julie Edwards', 'NULL', ['Reparto', 'Reparto'], ['su papel en Tratamiento real'], 'NULL'), ('Talia Lesser', 'NULL', ['Reparto', 'Reparto'], ['su papel en Tratamiento real'], 'NULL'), ('Ivan Stojanov', 'NULL', ['Reparto', 'Reparto'], ['su papel en Tratamiento real'], 'NULL'), ('An Xin Chang', 'NULL', ['Reparto', 'Reparto'], ['su papel en Tratamiento real'], 'NULL'), ('James Gaylyn', '1954', ['Reparto', 'Reparto'], ['X', 'Un puente hacia Terabithia', 'Avatar'], 'NULL'), ('Kube Jones-Neill', 'NULL', ['Reparto', 'Reparto'], ['su papel en Tratamiento real'], 'NULL'), ('Amanda Billing', '1976', ['Reparto', 'Cámara y electricidad', 'Arte'], ['Power Rangers Ninja Steel', 'Shortland Street', 'Sui Generis'], ['New Zealand Film and TV Awards (II) 2012 Nominado General TV Award Best Performance by an Actress Shortland Street', 'Bilbao Seriesland Festival 2022 Nominado Bilbao Seriesland Web Fest Festival Best Ensemble Cast Sui Generis Compartido con: Joe Dekkers-Reihana · Josh McKenzie · Buster Caldwell · Sam McKillen · Jack Henderson · Flynn Steward · · · · · · · · · · · · · · · · · · ·', 'NZ Web Fest 2022 Nominado Jury Award Best Ensemble: NZ Narrative Sui Generis Compartido con: Josh McKenzie · Buster Caldwell · Genevieve Mora · Joe Dekkers-Reihana · Jack Henderson · · · · · · · · · · · · · · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Chelsie Preston Crayford', '1987', ['Reparto', 'Dirección', 'Guion'], ['Underbelly', 'The Code', 'M3GAN'], ['Logie Awards 2012 Nominado Logie Most Popular New Female Talent Underbelly 2012 Ganador Graham Kennedy Award for Most Outstanding New Talent Underbelly', 'New Zealand Film and TV Awards (I) 2007 Ganador Film Award Best Short Film Performance Fog', 'Australian Academy of Cinema and Television Arts (AACTA) Awards 2015 Ganador AACTA Award Best Guest or Supporting Actress in a Television Drama The Code Episode 1', 'The Equity Ensemble Awards 2012 Nominado The Equity Award Outstanding Performance by an Ensemble in a Mini-series or Telemovie Underbelly Season 4: "Razor" Compartido con: John Batchelor · Matt Boesenberg · Richard Brancatisano · Jack Campbell · Khan Chittenden · · · · · · · · · · ·', 'NZ TV Awards 2022 Nominado Best Supporting Actress Best Supporting Actress The Panthers'])], [('Aaron Aguilera', '1977', ['Reparto', 'Especialistas', 'Edición'], ['Slammed', 'WWF SmackDown', 'WWE Raw'], 'NULL'), ('Dana Amromin', 'NULL', ['Reparto', 'Reparto'], ['su papel en Avenge', 'Sexxxting'], 'NULL'), ('Mark Arnold', '1957', ['Reparto', 'Producción', 'Reparto'], ['Blade Runner 2049', 'Teen Wolf (De pelo en pecho', 'Operación Kandahar'], 'NULL'), ('Chris Benton', '1981', ['Arte', 'Reparto', 'Arte'], ['At the End of the World', 'Avenge', 'I Love You, America'], 'NULL'), ('Michael Vincent Carrera', 'NULL', ['Reparto', 'Equipos adicionales', 'Dirección de casting'], ['su papel en Detectives Club', 'Alternate Universe: A Rescue Mission', 'The Stream'], 'NULL'), ('Dava Casoni', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Avenge', 'The Bilderberg Club: Meet the Shadow One World Government', 'Temps'], 'NULL'), ('Gregory Christian', 'NULL', ['Dirección', 'Guion', 'Producción'], ['su papel en Cowboy Junction', 'Avenge', 'Tamar'], 'NULL'), ('Steve Josefson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Black Hearts', 'Avenge', 'Cabin fever 2'], 'NULL'), ('Anne Lockhart', '1953', ['Reparto', 'Equipos adicionales', 'Sonido'], ['Más allá del tiempo', 'La conspiración del pánico', 'Daño colateral'], 'NULL'), ('Sebastian Martinez-Kadlecik', 'NULL', ['Reparto', 'Guion', 'Arte'], ['su papel en Quince, Betty I Am', 'Legend Hero'], 'NULL')], [('Jesse Holcomb', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Mourners, Inc'], 'NULL'), ('Brent Roberts', 'NULL', ['Reparto', 'Equipos adicionales', 'Guion'], ['su papel en We All Die Alone', 'Andrew Ginger/Likes Dogs', 'Future Ref'], ['San Diego Film Awards, US 2022 Ganador San Diego Film Award Best Ensemble Cast We All Die Alone Compartido con: Brian Patrick Butler · Frank DiPalermo · Joshua Alan Jones · Patrick Mayuyu · Durwood Murray · · ·', 'Oceanside International Film Festival, US 2022 Nominado Jury Award Best Supporting Actor We All Die Alone', 'Downtown Film Festival Los Angeles 2022 Ganador Festival Award Best Ensemble Cast in a Short Film We All Die Alone Compartido con: Brian Patrick Butler · Frank DiPalermo · Joshua Alan Jones · Patrick Mayuyu · Durwood Murray · · ·', 'Atlanta ShortsFest 2022 Nominado Jury Award Best Ensemble Cast We All Die Alone Compartido con: Brian Patrick Butler · Frank DiPalermo · Joshua Alan Jones · Patrick Mayuyu · Durwood Murray · · ·', 'Wildsound Feedback Writing and Film Festival 2021 Ganador Audience Award December Short Film Feedback LGBTQ+ Toronto Film Festival Best Performances We All Die Alone Compartido con: Brian Patrick Butler · Frank DiPalermo · Joshua Alan Jones · Patrick Mayuyu · Durwood Murray · · ·']), ('Angie Sarkisyan', 'NULL', ['Sonido', 'Reparto', 'Sonido'], ['Call of Duty: Modern Warfare III', 'Scott Pilgrim da el salto', 'Department of One'], 'NULL')], [('Luciana Dalfovo', 'NULL', ['Reparto', 'Reparto'], ['su papel en Maturità'], 'NULL'), ('Paula Ledesma', 'NULL', ['Reparto', 'Guion', 'Edición'], ['su papel en Todo el tiempo del mundo', 'Maturità', 'Tú decides'], 'NULL'), ('Matías Ludueña', 'NULL', ['Reparto', 'Cámara y electricidad', 'Reparto'], ['su papel en Minievo', 'Tres D', 'Maturità'], 'NULL'), ('Florencia Ozan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Maturità'], 'NULL'), ('Candela Álvarez', 'NULL', ['Reparto', 'Guion', 'Cámara y electricidad'], ['su papel en Maturità', 'Ciudad Universitaria'], 'NULL')], [('Valerie Rose Lohman', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en What Remains of Edith Finch', 'Wylde Flowers', 'Tower of God'], ['BAFTA Awards 2018 Nominado BAFTA Games Award Best Performer What Remains of Edith Finch', 'Young Entertainer Awards 2018 Nominado Young Entertainer Award Best Young Actress - Voice Over Role Raising a Rukus 2018 Ganador Young Entertainer Award Best Young Actress 16-21 - Short Film Intrepid', 'Five Continents International Film Festival 2019 Ganador Special Mention Lead Actress Short Film Short Film Awards She Comes in Colors Everywhere', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Erica Colarusso', 'NULL', ['Reparto', 'Reparto'], 'NULL', 'NULL'), ('Tate McMillan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bullpen', 'Gilly and Keeves: The Special', 'You Forgot Your Wallet?'], 'NULL'), ('Costadina Buelna', 'NULL', ['Reparto', 'Diseño de producción', 'Reparto'], ["su papel en Tomorrow's Game", "I Didn't Mean to Haunt You", "We'll Find A Way"], 'NULL'), ('Michael Sullivan', '1979', ['Reparto', 'Producción', 'Guion'], ['Anatomía de Grey', '9-1-1: Lone Star', 'Princess Rap Battle'], 'NULL')], [('Sayazhan Kulymbetova', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Crying Steppe'], 'NULL'), ('Akylkhan Almassov', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Crying Steppe', 'The Reporter', 'Others'], 'NULL'), ('Dulyga Akmolda', 'NULL', ['Reparto', 'Reparto'], ['su papel en Myn Bala', 'Rebirth Island y Zhanym, ty ne poverish'], 'NULL'), ('Sergey Ufimtzev', 'NULL', ['Reparto', 'Reparto'], ['su papel en Likvidator', 'Claustro', 'The Crying Steppe'], 'NULL'), ('Safuan Shaimerdenov', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dorogie moi deti', 'The Crying Steppe', 'ToyiHana'], 'NULL'), ('Roman Zhukov', 'NULL', ['Reparto', 'Guion', 'Sonido'], ['su papel en Paralympian', 'Claustro', 'The Crying Steppe'], 'NULL'), ('Adilet Zhangali', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Crying Steppe'], 'NULL'), ('Aibar Tangyt', 'NULL', ['Reparto', 'Reparto'], ['su papel en Big Little Life', 'The Crying Steppe', 'Kazakh Khanate - Golden Throne'], 'NULL'), ('Rayana Daulet', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Crying Steppe'], 'NULL'), ('Sagat Zhylgheldiyev', 'NULL', ['Reparto', 'Reparto'], ['Myn Bala', 'The Crying Steppe'], 'NULL')], [], [('Miao Xie', '1984', ['Reparto', 'Reparto'], ['Hung Hei Kwun: Siu Lam ng zou', 'Mi padre es un héroe', 'Da meng xi you'], 'NULL'), ('Enyou Yang', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ren sheng da shi', 'Give You A Candy y Jiao huan ren sheng'], ['Golden Rooster Awards 2022 Nominado Golden Rooster Best Actress Ren sheng da shi', 'Asian Film Awards 2023 Nominado Asian Film Award Best Newcomer Ren sheng da shi', 'Shanghai Film Critics Awards 2023 Nominado Shanghai Film Critics Award Best New Actress Ren sheng da shi', 'Huading Award 2023 Nominado Huading Award Best Actress in a Motion Picture Ren sheng da shi', "Guangzhou Student Film Festival 2022 Nominado Students' Choice Award Favorite Actress Ren sheng da shi"])], [('Ginger Goldbaum', 'NULL', ['Reparto', 'Reparto'], ["su papel en Vampire's Rage", 'Grabbing the Sun', 'Gsnaps: I Rap This'], 'NULL'), ('Jordan Cann', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Street Dreams: Los Angeles', 'From Paris to Rome', 'Straight Outta Compton'], 'NULL'), ('Michael Antonio', 'NULL', ['Reparto', 'Especialistas', 'Dirección de fotografía y cámara'], ['su papel en The Divinity Streak', 'Night of The Living Dead Girl', 'You Big Whale!'], 'NULL'), ('Doug Flores', 'NULL', ['Reparto', 'Reparto'], ['su papel en From Paris to Rome', 'My Brothers Keeper', "Vampire's Rage"], 'NULL'), ('Kaleila Jordan', 'NULL', ['Reparto', 'Reparto'], ['Red Camaro', "Angie's Cure", 'Dreaming Hollywood'], 'NULL'), ('Aaron Thompson', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Project Elementals', 'Let Me Explain', 'From Paris to Rome'], 'NULL')], [('Jaye Travis Brooks Sr.', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights', 'Return to Pennsic', 'U'], 'NULL'), ('Catlin Brooks', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Anthony De Longis', '1950', ['Reparto', 'Especialistas', 'Equipos adicionales'], ['Masters del universo', 'De profesión: duro', 'Indiana Jones y el reino de la calavera de cristal'], ['Screen Actors Guild Awards 2009 Nominado Actor Outstanding Performance by a Stunt Ensemble in a Motion Picture Indiana Jones y el reino de la calavera de cristal Compartido con: Don Abbatiello · Wade Allen · Greg Anthony · Jonathan Arthur · Matt Baker · Pat Banta · Helena Barrett · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Action on Film International Film Festival, USA 2015 Nominado Action on Film Award Breakout Action Star - Male - Short Boomerang Returns 2013 Nominado Action on Film Award Best Supporting Actor - Short Cowboy Creed', 'NAVGTR Awards 2012 Nominado NAVGTR Award Performance in a Comedy, Supporting Bulletstorm', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Paul Friedel', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Amy Graham', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Lance Hoffman', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Zorikh Lequidre', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Redemption of Captain Zorikh', 'American Knights', 'Passengers of 7D'], 'NULL'), ('Chris Minerd', 'NULL', ['Reparto', 'Dirección de fotografía y cámara', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Micah Nelson', 'NULL', ['Reparto', 'Reparto'], ['su papel en American Knights'], 'NULL'), ('Stephen Parker', 'NULL', ['Reparto', 'Dirección de fotografía y cámara', 'Reparto'], ['su papel en American Knights', 'Harlem Knight Fight'], 'NULL')], [('Alex Désert', 'NULL', ['Reparto', 'Producción', 'Banda sonora'], 'NULL', 'NULL'), ('Travis Myers', 'NULL', ['Reparto', 'Guion', 'Arte'], ['su papel en J'], 'NULL'), ('Christopher Amitrano', '1973', ['Reparto', 'Producción', 'Guion'], ['El abogado del Lincoln', 'All Rise', 'Urgencias'], 'NULL'), ('Sharzad Kiadeh', 'NULL', ['Maquillaje', 'Reparto', 'Maquillaje'], ['su papel en The Kitchen', 'JustBoobs Sketch'], 'NULL')], [('Hazar Deniz BG', '1998', ['Dirección', 'Guion', 'Reparto'], 'NULL', 'NULL'), ('Ali Meftun Zade', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared'], 'NULL'), ('Ramazan Uygur', 'NULL', ['Reparto', 'Casting', 'Dirección de casting'], ['su papel en The Cruise', 'Run Away If You Scared', 'The Guest : Misafir'], 'NULL'), ('Bahadir Kutluay', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared', 'Araz'], 'NULL'), ('Halime Karatas', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared'], 'NULL'), ('Beyza AK', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared'], 'NULL'), ('Halil Cogul', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared'], 'NULL'), ('Hacer Nur Kaplan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared', 'Araz'], 'NULL'), ('Ahmet Kara', 'NULL', ['Reparto', 'Reparto'], ['su papel en Run Away If You Scared'], 'NULL')], [('Emanuel Coolwijk', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mime', "Barry's Eerste Deal"], 'NULL'), ('Thij de Haan', 'NULL', ['Reparto', 'Guion', 'Equipos adicionales'], ['su papel en I Can Fly', 'Mime', 'Dutchtown'], 'NULL'), ('Carline Dedden', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mime'], 'NULL'), ('Joep Drewes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Molly', 'Mime'], 'NULL'), ('Wart Frencken', 'NULL', ['Reparto', 'Guion y continuidad', 'Reparto'], ['su papel en De Rotter Dwang Academy', 'Buddy', 'Mime'], 'NULL'), ('Tamara Friedman', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en The Codfather', 'De Bitterbal', 'Diary of a Madman'], 'NULL'), ('Marit Golstein', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mime'], 'NULL'), ('Jop Gorris', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Mime', 'Negeri Van Oranje', 'Providentia'], 'NULL'), ('Kumar Heggelman', 'NULL', ['Reparto', 'Reparto'], ['su papel en De Schoonmakers', 'Weerzien', 'Mime'], 'NULL'), ('Liza Hein', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mime', 'Peer'], 'NULL')], [('Micaela Wittman', '1999', ['Reparto', 'Guion', 'Producción'], ['Remy & Arletta', 'Teen Wolf', 'Clairevoyant'], 'NULL'), ('Anousha', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en El agente nocturno', 'Lifeline', 'Clairevoyant'], 'NULL'), ('Amy Benedict', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Sneakers (Los fisgones', 'Treintaytantos', 'Anatomía de Grey'], 'NULL'), ('Arthur De Larroche', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Remy & Arletta', 'Clairevoyant', 'American Bistro'], 'NULL'), ('Sam Dissanayake', 'NULL', ['Reparto', 'Reparto'], ['su papel en Amor en juego', 'The Greasy Strangler', 'Si fuera fácil'], 'NULL'), ('Heidi Fecht', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Valiant', 'The Picture of You', 'Ángel'], 'NULL'), ('Rhys Hyatt', 'NULL', ['Reparto', 'Sonido', 'Reparto'], ['su papel en Clairevoyant', "Leave 'em Laughing", 'MapleStory 2'], 'NULL'), ('Andrew Kirkley', 'NULL', ['Reparto', 'Producción', 'Equipos adicionales'], ['su papel en A House Is Not a Disco', 'The Outlier', 'The Last'], 'NULL'), ('Ashley Moret', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dead in Texas', 'The Sad Cafe', "Je T'aime, Au Revoir"], 'NULL')], [('Carolina Crescentini', '1980', ['Reparto', 'Reparto'], ['Tengo algo que deciros', '20 sigarette', 'Háblame de amor'], ['David di Donatello Awards 2008 Nominado David Best Supporting Actress (Migliore Attrice non Protagonista) Háblame de amor', 'Giffoni Film Festival 2009 Ganador Giffoni Award', "Italian National Syndicate of Film Journalists 2021 Nominado Silver Ribbon Best Supporting Actress (Migliore Attrice Non Protagonista) La bambina che non voleva cantare 2012 Nominado Silver Ribbon Best Actress (Migliore Attrice Protagonista) L'industriale 2011 Ganador Silver Ribbon Best Supporting Actress (Migliore Attrice Non Protagonista) 20 sigaretteBoris - Il film 2009 Nominado Silver Ribbon Best Supporting Actress (Migliore Attrice Non Protagonista) Due partite Compartido con: Margherita Buy · Paola Cortellesi · Isabella Ferrari · Marina Massironi · Valeria Milillo · Claudia Pandolfi · 2008 Nominado Silver Ribbon Best Actress (Migliore Attrice Protagonista) Cemento armatoI demoni di San Pietroburgo", "Venice Film Festival 2010 Ganador Pasinetti Award - Special Mention 20 sigarette To the cast and the producers of the film. Compartido con: Giorgio Colangeli · Vinicio Marchioni · Tilde Corsi · Gianni Romoli · Claudio Bonivento 2009 Ganador L'Oreal Paris Award", 'Capri, Hollywood 2007 Ganador Capri Exploit Award Cemento armato']), ('Claudio Gioè', '1975', ['Reparto', 'Reparto'], ['Ti racconto una storia', 'Passato prossimo', 'La mafia sólo mata en verano'], ['Giffoni Film Festival 2012 Ganador Giffoni Award', 'Italian National Syndicate of Film Journalists 2003 Nominado Silver Ribbon Best Supporting Actor (Migliore Attore Non Protagonista) Passato prossimo Compartido con: Claudio Santamaria', 'Cittadella del Corto 2005 Ganador Best Actor Ti racconto una storia', 'Salerno Shadowline Film Festival 2008 Ganador Shadowline Award', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Kokou Aurelien Gaya', 'NULL', ['Reparto', 'Equipos adicionales', 'Guion'], ['su papel en Valerian y la ciudad de los mil planetas', 'First Man', 'You Hurt I Hurt'], 'NULL'), ('Pietro De Silva', '1954', ['Reparto', 'Dirección', 'Reparto'], ['La vida es bella', 'El capo de Corleone', 'Amor embrujado'], 'NULL'), ('Paolo Sassanelli', '1958', ['Reparto', 'Dirección', 'Guion'], ['Uerra', 'Ammore', 'Rosso come il cielo'], ['Cork International Film Festival 2013 Nominado Grand Prix International Ammore', 'David di Donatello Awards 2013 Nominado David Best Short Film (Migliore Cortometraggio) Ammore 2010 Nominado David Best Short Film (Migliore Cortometraggio) Uerra', 'Isfahan International Festival of Films for Children & Young Adults 2008 Ganador Golden Butterfly Best Screenplay Rosso come il cielo Compartido con: Cristiano Bortone · Monica Zapelli', "Italian National Syndicate of Film Journalists 2014 Nominado Silver Ribbon Best Short Film Director (Regista del Miglior Cortometraggio) Ammore 2014 Ganador Silver Ribbon Best Supporting Actor (Migliore Attore Non Protagonista) Song 'e Napule Compartido con: Carlo Buccirosso 2009 Ganador Special Mention Short Film (Cortometraggio) Uerra 2009 Ganador Special Mention Short Film (Cortometraggio) La preda", 'Manhattan Short Film Festival 2010 Ganador Grand Prize Short Film Uerra']), ('Michele Riondino', '1979', ['Reparto', 'Dirección', 'Guion'], ['Palazzina Laf', 'Bella addormentata', 'Il passato è una terra straniera'], ['Berlin International Film Festival 2010 Ganador EFP Shooting Star Italy.', 'David di Donatello Awards 2024 Ganador David Best Actor (Migliore Attore Protagonista) Palazzina Laf 2024 Nominado David Best New Director (Migliore Regista Esordiente) Palazzina Laf 2024 Nominado David Best Original Screenplay (Migliore Sceneggiatura Originale) Palazzina Laf Compartido con: Maurizio Braucci 2017 Nominado David Best Actor (Migliore Attore Protagonista) La ragazza del mondo', 'Giffoni Film Festival 2010 Ganador Giffoni Award', 'Italian National Syndicate of Film Journalists 2017 Nominado Silver Ribbon Best Actor (Migliore Attore Protagonista) La ragazza del mondo 2013 Nominado Silver Ribbon Best Supporting Actor (Migliore Attore Non Protagonista) Bella addormentataAcciaio 2012 Nominado Silver Ribbon Best Supporting Actor (Migliore Attore Non Protagonista) Gli sfiorati 2011 Ganador Silver Ribbon of the Year Noi credevamo Compartido con: Luigi Lo Cascio · Valerio Binasco · Francesca Inaudi · Luigi Pisani · Edoardo Natoli · Andrea Bosca · · · · · · · · · · · · · · · · · · 2010 Ganador Guglielmo Biraghi Award Dieci inverni 1 más', 'Miami Film Festival 2009 Ganador Special Mention Best Actor - World Competition Il passato è una terra straniera']), ('Alfonso Santagata', '1940', ['Reparto', 'Reparto'], ['Gomorra', 'Vaselina roja', 'Il conte di Melissa'], 'NULL'), ('Eriq Ebouaney', '1967', ['Reparto', 'Reparto'], ['3 días para matar', 'El reino de los cielos', 'Transporter 3'], 'NULL'), ('Dino Abbrescia', '1966', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Reparto'], ['Manual de amor', '2061: Un anno eccezionale', 'Intelligence - Servizi & segreti'], 'NULL'), ('David Coco', '1970', ['Reparto', 'Reparto'], ["L'uomo di vetro", 'Sposami', 'Violeta como el mar'], 'NULL')], [('Max Engelke', '1982', ['Reparto', 'Reparto'], ['Verbotene Liebe', 'Freundschaft mit Herz', 'Da kommt Kalle'], 'NULL'), ('Nic Romm', '1974', ['Reparto', 'Dirección', 'Guion'], ['Anonymous', 'Vicky el vikingo', 'Namosh: Cold Cream'], 'NULL'), ('Nora Tschirner', '1981', ['Reparto', 'Producción', 'Guion'], ['Un conejo sin orejas', 'En el lugar del crimen', 'Kebab Connection'], ['Bambi Awards 2008 Ganador Bambi Film - National Un conejo sin orejas Compartido con: Til Schweiger', 'Málaga Spanish Film Festival 2010 Ganador Special Mention Best Actress (Mejor Actriz) Bon appétit', 'Romy Gala, Austria 2019 Nominado Romy Favorite Actress in a Series (Beliebtester weiblicher Serienstar) En el lugar del crimen 2015 Ganador Romy Favorite Actress in a Series (Beliebtester weiblicher Serienstar) En el lugar del crimen', 'German Comedy Awards 2008 Ganador German Comedy Award Best Feature Film Comedy (Beste Filmkomödie) Un conejo sin orejas Compartido con: Til Schweiger 2008 Ganador German Comedy Award Best Actress in a Comedy (Beste Schauspielerin in einer Comedy) Un conejo sin orejas', 'Nuremberg Film Festival "Turkey-Germany" 2013 Ganador Öngören Prize Waiting Area Compartido con: Natalie Beer 2005 Ganador Best Actress Kebab Connection']), ('Jakob Benkhofer', '1986', ['Reparto', 'Reparto'], ['Jack Taylor', 'Nullpunkt', 'Nord bei Nordwest'], 'NULL'), ('Susanne Maierhöfer', 'NULL', ['Reparto', 'Reparto'], ['su papel en En el lugar del crimen', 'Nullpunkt', 'Die Vermissten'], 'NULL')], [('Auðunn Blöndal', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Cop Secret', 'Svínasúpan', 'Steypustöðin'], 'NULL'), ('Egill Einarsson', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Cop Secret', "Black's Game", 'Ævintýri á einkamál'], 'NULL'), ('Sverrir Þór Sverrisson', '1977', ['Reparto', 'Guion', 'Producción'], ['Cop Secret', 'Algjör Sveppi og Gói bjargar málunum', 'Algjör Sveppi og leitin að Villa'], 'NULL'), ('Steinunn Ólína Þorsteinsdóttir', '1969', ['Reparto', 'Reparto'], ['Case', 'Prisioneras', 'Atrapados'], 'NULL'), ('Björn Hlynur Haraldsson', '1974', ['Reparto', 'Guion', 'Dirección'], ['Homecoming', 'Me estás matando Susana'], ['Berlin International Film Festival 2006 Ganador EFP Shooting Star Iceland.', 'Chicago International Film Festival 2015 Nominado Gold Hugo New Directors Competition Blóðberg', 'Edda Awards, Iceland 2024 Ganador Edda Award Supporting Actor of the Year (Leikari ársins í aukahlutverki) Villibráð 2023 Ganador Edda Award Screenplay of the Year (Handrit ársins) Blackport Compartido con: Gísli Örn Garðarsson · Mikael Torfason 2023 Nominado Edda Award Director of the Year (Leikstjóri ársins) Blackport Compartido con: María Reyndal · Gísli Örn Garðarsson 2022 Ganador Edda Award Supporting Actor of the Year (Leikari ársins í aukahlutverki) Lamb 2022 Nominado Edda Award Supporting Actor of the Year (Leikari ársins í aukahlutverki) Cop Secret 5 más', 'São Paulo International Film Festival 2015 Nominado International Jury Award Best Feature Film Blóðberg', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Vivian Ólafsdóttir', 'NULL', ['Reparto', 'Reparto'], ['su papel en Operación Napoleón', 'Cop Secret', 'Axlar-Björn'], 'NULL'), ('Rúrik Gíslason', '1988', ['Reparto', 'Reparto'], ['Cop Secret', 'Bibi & Tina: Einfach Anders', 'Húsó'], 'NULL'), ('Steinþór Hróar Steinþórsson', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Buenos vecinos', 'Þorsti', 'Steypustöðin'], 'NULL'), ('Gunnar Hansson', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Bakk', 'I Make Sense of Humour', 'Beowulf & Grendel: el retorno de la bestia'], 'NULL'), ('Birgir Gíslason', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cop Secret', 'Eurogarðurinn'], 'NULL')], [('Fred Ewanuick', '1971', ['Reparto', 'Producción', 'Guion'], ['Sólo amigos', 'Corner Gas: The Movie', 'Corner Gas'], ['Canadian Comedy Awards 2005 Nominado Canadian Comedy Award Television - Pretty Funny Male Performance Corner Gas', 'Gemini Awards 2007 Nominado Gemini Best Performance by an Actor in a Featured Supporting Role in a Dramatic Series Robson Arms For episode "Saultology". 2007 Ganador Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Gopher It". Compartido con: Brent Butt · Lorne Cardinal · Gabrielle Miller · Eric Peterson · Nancy Robertson · Tara Spencer-Nairn · 2006 Nominado Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Merry Gasmas". Compartido con: Brent Butt · Lorne Cardinal · Gabrielle Miller · Eric Peterson · Nancy Robertson · Tara Spencer-Nairn · 2004 Nominado Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Face Off" Compartido con: Brent Butt · Lorne Cardinal · Gabrielle Miller · Eric Peterson · Nancy Robertson · Tara Spencer-Nairn ·', 'Leo Awards 2019 Nominado Leo Best Performance in an Animation Program or Series Shop Class 2011 Nominado Leo Best Performance or Host(s) in a Music, Comedy or Variety Program or Series Dan for Mayor For episode "Mayor for a Day". 2008 Nominado Leo Best Supporting Performance by a Male in a Feature Length Drama Crossing 2006 Nominado Leo Best Performance or Host in a Music, Comedy or Variety Program or Series Corner Gas For episode "Wedding Card" 2005 Nominado Leo Music, Comedy, or Variety Program or Series: Best Performance or Host(s) Corner Gas For episode "Cell Phone".', 'Peñíscola Comedy Film Festival 2004 Ganador Best Actor The Delicate Art of Parking', 'Behind the Voice Actors Awards 2016 Nominado BTVA Television Voice Acting Award Best Vocal Ensemble in a New Television Series Dinotrux Compartido con: Andrew Francis · Richard Ian Cox · Ashleigh Ball · Matt Hill · Brian Drummond · Cree Summer · · ·']), ('Nancy Robertson', 'NULL', ['Reparto', 'Producción', 'Guion'], 'NULL', ['Canadian Comedy Awards 2011 Ganador Canadian Comedy Award Best Performance by a Female - Television Caution: May Contain NutsCásate conmigoNadie es perfectoHiccupsLittle Mosque on the Prairie Compartido con: Aimée Beaudoin · Angela Asher · Brooke Palsson · Sitara Hewitt 2006 Nominado Canadian Comedy Award Best Performance by a Female - Television Corner Gas 2005 Nominado Canadian Comedy Award Television - Pretty Funny Female Performance Corner Gas', 'Gemini Awards 2007 Ganador Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Gopher It". Compartido con: Brent Butt · Lorne Cardinal · Fred Ewanuick · Gabrielle Miller · Eric Peterson · Tara Spencer-Nairn · 2006 Nominado Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Merry Gasmas". Compartido con: Brent Butt · Lorne Cardinal · Fred Ewanuick · Gabrielle Miller · Eric Peterson · Tara Spencer-Nairn · 2004 Nominado Gemini Best Ensemble Performance in a Comedy Program or Series Corner Gas For episode "Face Off" Compartido con: Brent Butt · Lorne Cardinal · Fred Ewanuick · Gabrielle Miller · Eric Peterson · Tara Spencer-Nairn ·', 'Leo Awards 2021 Ganador Leo Best Performance in an Animation Series Corner Gas Animated 2012 Ganador Leo Best Music, Comedy or Variety Program or Series Hiccups Compartido con: Brent Butt · David Storey · Laura Lightbown · Arvi Liimatainen 2012 Nominado Leo Best Performance or Host(s) in a Music, Comedy, or Variety Program or Series Hiccups 2011 Ganador Leo Best Music, Comedy, or Variety Program or Series Hiccups Compartido con: Brent Butt · David Storey · Laura Lightbown · Arvi Liimatainen 2011 Ganador Leo Best Performance or Host(s) in a Music, Comedy or Variety Program or Series Hiccups For episode "Hot Luv". 6 más', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Brendan Taylor', 'NULL', ['Reparto', 'Arte', 'Diseño de producción'], ['su papel en El baile de las luciérnagas', 'Sobrenatural', 'Fargo'], 'NULL'), ('Nikki Chohan', 'NULL', ['Reparto', 'Reparto'], ['su papel en So Help Me Todd', 'Lucky Hank', 'Valley of the Boom'], 'NULL'), ('Jamie Taylor', 'NULL', ['Reparto', 'Reparto'], ['su papel en Baleful', 'Last Night', 'A New Leash on Life'], 'NULL'), ('Sarah Faye Bernstein', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Reparto'], ['su papel en Last Night', 'Baleful', 'Sensitive Parts'], 'NULL'), ('Dylan Rose Marshall', 'NULL', ['Reparto', 'Reparto'], ['su papel en A New Leash on Life'], 'NULL')], [], [('Roy Abramsohn', 'NULL', ['Reparto', 'Sonido', 'Reparto'], ['su papel en Escape from Tomorrow', 'Dying to Sleep', 'Searching'], ['Idyllwild International Festival of Cinema 2018 Nominado IIFC Award Best Supporting Actor - Short Swim 2018 Nominado IIFC Award Best Ensemble Cast- short film or web-series Swim Compartido con: Katharine Lee McEwan · Susan Papa · Gavin Fink · Jen Richards', 'Houston Comedy Film Festival 2021 Ganador Best Ensemble Cast Best Ensemble Cast HOW WE DO - Sketches of the Species Compartido con: Pete Macnamara · Denise Reed · Jabari Gray · Kerri Louise · Lina Hoang · Darius Levanté · Glenn Speers ·', 'IndieFEST Film Awards 2022 Ganador Award of Excellence Best Ensemble Cast Dying to Sleep Compartido con: SarahLydia Sophia · Paris Dylan · Victoria Baldesarra · Dar Dixon · Maria Pinsent · Jataun Gilbert · · ·', 'Indie Short Fest 2022 Nominado December Award Best Ensemble Cast HOW WE DO - Sketches of the Species Compartido con: Jabari Gray · Glenn Speers · Lina Hoang · Brandon Brown · Hannah Beck · Shelley Valfer · Kerri Louise', 'SoCal Film Awards 2023 Nominado Additional Award Best Ensemble Cast Dying to Sleep Compartido con: Jataun Gilbert · Alex Keever · Maria Pinsent · SarahLydia Sophia · Victoria Baldesarra · Dave Sheridan · · ·']), ('Meg Cionni', '1984', ['Reparto', 'Producción', 'Guion'], ['Misterio en Amán', 'Waking', 'The Wedding Party'], 'NULL'), ('Bree Doehring', 'NULL', ['Equipos adicionales', 'Producción', 'Edición'], ['su papel en Jay y Bob el silencioso: el reboot', 'Alice in Somnia', 'Vestige'], 'NULL'), ('Kobi Libii', '1980', ['Reparto', 'Guion', 'Dirección'], ['The American Society of Magical Negroes', 'Alpha House', 'Madam Secretary'], 'NULL'), ('Riley Polanski', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Alien Abduction', 'Worthless Words', 'Eres lo peor'], 'NULL')], [('Kelly Vrooman', 'NULL', ['Reparto', 'Guion', 'Equipos adicionales'], ['su papel en Black-ish', 'Victor Crowley', 'The Chica Show'], 'NULL'), ('Nick Creegan', '1993', ['Reparto', 'Producción', 'Reparto'], ['Ley y orden: Crimen organizado', 'Batwoman', 'Good Egg'], 'NULL'), ('Don Priess', '1957', ['Guion', 'Producción', 'Edición'], ['My Mom and the Girl', 'I Love Lucifer', 'I Love Lucifer'], ["Cleveland International Film Festival 2017 Ganador Programmers' Choice Award - In Celebration of the life of Toni Oliverio My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti", 'Best Shorts Competition 2016 Ganador Award of Excellence Special Mention Women Filmmakers My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti', 'Connect Film Festival, AUS 2016 Nominado Jury Prize Bechdel Award for Storytelling My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti 2016 Nominado Golden Horse Best Comedy My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti 2016 Nominado Jury Prize Social Impact Award My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti 2016 Nominado Jury Prize Best Picture (Best Director, Best Writer & Best Producer) My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti 2016 Ganador Jury Prize Best Picture (Best Director, Best Writer & Best Producer) My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti', 'Indie Series Awards 2023 Nominado Indie Series Award Best Audio Fiction Series I Love Lucifer Compartido con: Susie Singer Carter', 'Emerging Filmmakers Film Festival 2017 Nominado Honorable Mention Best Short LGBTQ My Mom and the Girl Compartido con: Susie Singer Carter · Romell Foster-Owens · Tony Cacciotti'])], [('George Dippold', 'NULL', ['Reparto', 'Reparto'], ['su papel en Shooting the Prodigal', "People Don't Die in the Comics Anymore", 'Tree Hugger'], 'NULL'), ('Alescia Jester', 'NULL', ['Reparto', 'Reparto'], ['su papel en Half the Battle'], 'NULL'), ('Aleshia Jester', 'NULL', ['Reparto', 'Reparto'], ['su papel en Half the Battle'], 'NULL'), ('Artevia Jester', 'NULL', ['Reparto', 'Reparto'], ['su papel en Half the Battle'], 'NULL'), ('Charles Jester', 'NULL', ['Reparto', 'Reparto'], ['su papel en Half the Battle'], 'NULL'), ('Scott Rich', 'NULL', ['Reparto', 'Reparto'], ['su papel en Half the Battle'], 'NULL'), ('Jessica Anne Taylor', 'NULL', ['Reparto', 'Casting', 'Equipos adicionales'], ['su papel en Compassionate Release', 'Inexplicable Epiphanies', 'Rewired'], 'NULL')], [('Carla Kinzo', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Guion'], ['su papel en Quando Eu Era Vivo', 'Nascemos Hoje, Quando o Céu Estava Carregado de Ferro e Veneno', 'Dueto Cego'], 'NULL'), ('Otto Jr.', '1971', ['Reparto', 'Reparto'], ['Tropa de élite', 'Todos los días la misma noche', 'Túnel de corrupción'], 'NULL'), ('Rômulo Braga', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Carvão', 'Elon Não Acredita na Morte', 'Sol'], ['Brazilia Festival of Brazilian Cinema 2016 Ganador Candango Trophy Best Actor (Melhor Ator) Elon Não Acredita na Morte', 'Gramado Film Festival 2019 Ganador Golden Kikito Short Film Competiton 35mm - Best Actor (Curta Metragem em 35mm - Melhor Ator) Marie', 'Rio de Janeiro International Film Festival 2021 Ganador Première Brazil Best Actor (Melhor Ator) Sol 2014 Ganador Première Brazil Best Supporting Actor (Melhor Ator Coadjuvante) Sangue Azul', 'CinEuphoria Awards 2018 Nominado CinEuphoria Best Ensemble - National Competition Joaquim Compartido con: Welket Bungué · Nuno Lopes · Julio Machado · Miguel Pinheiro · Isabél Zuaa', 'Prêmio Guarani 2023 Nominado Premio Guarani Best Actor Sol 2023 Ganador Premio Guarani Best Supporting Actor Carvão 2021 Nominado Premio Guarani Best Actor O Barco']), ('David Lobo', 'NULL', ['Reparto', 'Reparto'], ['su papel en Três Tigres Tristes', 'Desterro'], 'NULL'), ('Sara Antunes', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Segunda Chamada', 'De Dora, por Sara', 'Deslembro'], 'NULL'), ('Ricardo Calil', 'NULL', ['Guion', 'Dirección', 'Reparto'], ['su papel en Cine Marrocos', 'Uma Noite em 67', 'Eu Sou Carlos Imperial'], ['Cinema Brazil Grand Prize 2022 Nominado Cinema Brazil Grand Prize Best Documentary (Melhor Documentário) Cine Marrocos 2017 Nominado Cinema Brazil Grand Prize Best Documentary (Melhor Documentário) Eu Sou Carlos Imperial Compartido con: Renato Terra 2011 Nominado Cinema Brazil Grand Prize Best Documentary (Melhor Documentário) Uma Noite em 67 Compartido con: Renato Terra', 'Guadalajara International Film Festival 2019 Ganador Ibero-America Competition Feature Length Documentary Cine Marrocos Awarded with $150,000 Mexican Pesos', 'Leipzig DOK Festival 2018 Ganador Golden Dove Next Masters Competition Cine Marrocos', 'Prêmio Contigo Cinema, Brazil 2011 Nominado Jury Award Best Director - Documentary (Melhor Diretor - Documentário) Uma Noite em 67 Compartido con: Renato Terra', 'ACIE Awards, Brazil 2011 Ganador ACIE Award Best Documentary (Melhor Documentário) Uma Noite em 67 Compartido con: Renato Terra']), ('Bárbara Colen', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], ['su papel en Bacurau', 'Fogaréu', 'Doña Clara'], ['Cinema Brazil Grand Prize 2020 Nominado Cinema Brazil Grand Prize Best Actress (Melhor Atriz) Bacurau', 'Havana Film Festival New York 2022 Ganador Havana Star Best Actress Fogaréu', 'Prêmio Guarani 2021 Nominado Premio Guarani Best Supporting Actress Breve Miragem de Sol 2020 Nominado Premio Guarani Best Actress Bacurau', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Juliana Carneiro da Cunha', '1949', ['Reparto', 'Reparto'], ['LavourArcaica', 'Eduardo e Mônica', 'O Veneno da Madrugada'], ['Brazilia Festival of Brazilian Cinema 2001 Ganador Candango Trophy Best Supporting Actress (Melhor Atriz Coadjuvante) LavourArcaica', 'Cinema Brazil Grand Prize 2002 Ganador Cinema Brazil Grand Prize Best Actress (Melhor Atriz) LavourArcaica', 'Gramado Film Festival 2009 Ganador Golden Kikito Short Film Competition - Best Actress (Curta-Metragem em 35mm - Melhor Atriz) O Teu Sorriso', 'São Paulo Association of Art Critics Awards 2002 Ganador APCA Trophy Best Actress (Melhor Atriz) LavourArcaica', 'Prêmio Contigo Cinema, Brazil 2007 Nominado Jury Award Best Supporting Actress (Melhor Atriz Coadjuvante) O Veneno da Madrugada']), ('Hu Xiau Den', 'NULL', ['Reparto', 'Reparto'], ['su papel en Desterro'], 'NULL')], [('Peter Franzén', '1971', ['Reparto', 'Dirección', 'Guion'], ['Harjunpää & pahan pappi', 'Tumman veden päällä', 'Caza al asesino'], ['Jussi Awards 2014 Nominado Jussi Best Leading Actor (Paras miespääosa) Kerron sinulle kaiken 2013 Nominado Jussi Best Supporting Actor (Paras miessivuosa) Tie pohjoiseen 2011 Nominado Jussi Best Leading Actor (Paras miespääosa) Harjunpää & pahan pappi 2010 Ganador Jussi Best Supporting Actor (Paras miessivuosa) Rööperi 2005 Ganador Jussi Best Leading Actor (Paras miespääosa) Koirankynnen leikkaaja 3 más', 'Palm Springs International Film Festival 2014 Nominado New Voices/New Visions Grand Jury Prize Tumman veden päällä', 'Rouen Nordic Film Festival 2000 Ganador Best Actor Emboscada', 'Zlín International Film Festival for Children and Youth 2014 Nominado Best European Debut Film Tumman veden päällä', 'Brussels International Independent Film Festival 2003 Ganador Best Actor International Competition Pahat pojat Compartido con: Niko Saarela · Lauri Nurkse · Jasper Pääkkönen · Vesa-Matti Loiri']), ('Irina Björklund', '1973', ['Reparto', 'Música', 'Guion'], ['Peacemaker', 'El americano', 'Apeiron'], ['Berlin International Film Festival 2004 Ganador EFP Shooting Star Finland.', 'Jussi Awards 2002 Ganador Jussi Best Leading Actress (Paras naispääosa) Minä ja Morrison 2000 Nominado Jussi Best Leading Actress (Paras naispääosa) Emboscada', 'Tampere Film Festival 1997 Ganador Prize of the Youth Jury Finnish Short Film Under 30 Minutes For actress who has talent to perform in wide scale', 'Venla Awards 2007 Nominado Venla Best Actress (Paras naisnäyttelijä) Isabella', 'Kultainen Venla 2021 Nominado Kultainen Venla Best Actress (Paras naisnäyttelijä) Peacemaker 2014 Nominado Kultainen Venla Best Actress (Paras naisnäyttelijä) Apeiron 2013 Nominado Kultainen Venla Best Actress (Paras naisnäyttelijä) Inget ljus i tunneln']), ('Sampo Sarkola', '1978', ['Reparto', 'Equipos adicionales', 'Banda sonora'], ['Ikitie', 'Bordertown', 'Harjunpää & pahan pappi'], 'NULL'), ('Jenni Banerjee', '1981', ['Reparto', 'Reparto'], ['Harjunpää & pahan pappi', 'Joensuun Elli', 'Keihäsmatkat'], 'NULL'), ('Rosa Salomaa', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Casting'], ['su papel en Ikitie', 'Oma maa', 'Harjunpää & pahan pappi'], 'NULL'), ('Niilo Syväoja', '1991', ['Reparto', 'Banda sonora', 'Reparto'], ['Harjunpää & pahan pappi', 'Tummien perhosten koti', 'Helmiä ja sikoja'], 'NULL'), ('Jorma Tommila', '1959', ['Reparto', 'Guion', 'Arte'], ['Sisu', 'Un cuento gamberro de Navidad', 'Joulubileet'], 'NULL'), ('Ville Virtanen', '1961', ['Reparto', 'Dirección', 'Guion'], ['Sauna', 'Paha perhe', 'Karjalan kunnailla'], ['Guldbagge Awards 2021 Nominado Guldbagge Best Supporting Actor (Bästa manliga biroll) Den längsta dagen 2011 Nominado Guldbagge Best Supporting Actor (Bästa manliga biroll) Svinalängorna', 'Jussi Awards 2011 Ganador Jussi Best Leading Actor (Paras miespääosa) Paha perhe 2009 Nominado Jussi Best Leading Actor (Paras miespääosa) Sauna', 'Venla Awards 2003 Ganador Venla Best Actor (Paras miesnäyttelijä) Kylmäverisesti sinun', 'Kultainen Venla 2017 Ganador Kultainen Venla Best Actor (Paras miesnäyttelijä) Bordertown', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Tommi Korpela', '1968', ['Reparto', 'Guion', 'Música'], ['Un cuento gamberro de Navidad', 'Häiriötekijä', 'Putoavia enkeleitä'], ['Jussi Awards 2023 Nominado Jussi Best Supporting Role (Paras sivuosa) Kupla 2021 Nominado Jussi Best Supporting Actor (Paras miessivuosa) Viimased 2018 Nominado Jussi Best Leading Actor (Paras miespääosa) Ikitie 2016 Ganador Jussi Best Leading Actor (Paras miespääosa) Häiriötekijä 2009 Ganador Jussi Best Leading Actor (Paras miespääosa) Putoavia enkeleitä 1 más', 'Moscow International Film Festival 2019 Ganador Silver St. George Best Actor Tyhjiö', 'Marrakech International Film Festival 2007 Ganador Best Actor Miehen työ', 'Venla Awards 2006 Ganador Venla Best Actor (Paras näyttelijä) Firma 2004 Nominado Venla Best Actor (Paras miesnäyttelijä) Remontti', 'Kultainen Venla 2020 Ganador Kultainen Venla Writer of the Year (Vuoden käsikirjoittaja) M/S Romantic Compartido con: Jani Volanen']), ('Maria Järvenhelmi', '1975', ['Reparto', 'Música', 'Banda sonora'], ['Rölli ja metsänhenki', 'Vares - Pimeyden tango', 'Vares - Huhtikuun tytöt'], 'NULL')], [('Ali Suliman', '1977', ['Reparto', 'Producción', 'Reparto'], ['La sombra del reino', 'Red de mentiras', 'El único superviviente'], ['Antalya Golden Orange Film Festival 2020 Ganador International Feature Film Competition Award Best Actor 200 Meters', 'San Sebastián International Film Festival 2012 Nominado Silver Seashell Best Actor El atentado', 'Seattle International Film Festival 2013 Nominado Golden Space Needle Award Best Actor El atentado', 'Alexandria International Film Festival 2015 Ganador Best Actor Mars at Sunrise', 'Marrakech International Film Festival 2012 Nominado Best Actor El atentado']), ('Reymonde Amsellem', '1978', ['Reparto', 'Guion', 'Reparto'], ['Siete bendiciones', 'Expediente Anwar', 'My Lovely Sister'], 'NULL'), ('Evgenia Dodina', '1964', ['Reparto', 'Banda sonora', 'Reparto'], ['Shavua ve Yom', 'Lo Roim Alaich', 'Killing Eve'], ['Haifa International Film Festival 2011 Ganador Israeli Film Competition Award Best Actress Lo Roim Alaich', 'Awards of the Israeli Film Academy 2018 Nominado Award of the Israeli Film Academy Best Supporting Actress Vierges 2016 Nominado Award of the Israeli Film Academy Best Actress Shavua ve Yom 2009 Nominado Award of the Israeli Film Academy Best Supporting Actress Haiu Leilot 2001 Nominado Award of the Israeli Film Academy Best Actress Made in Israel 1998 Nominado Award of the Israeli Film Academy Best Actress Zirkus Palestina 1 más', 'Oldenburg Film Festival 2016 Ganador Seymour Cassel Award Outstanding Performance by an Ensemble Shavua ve Yom Compartido con: Shai Avivi · Sharon Alexander · Uri Gavriel · Carmit Mesilati Kaplan · Tomer Capone', 'Awards of the Israeli Television Academy 2023 Nominado Award of the Israeli Television Academy Best Supporting Actress in a Comedy Series Sovietzka 2003 Nominado Award of the Israeli Television Academy Best Acting in a Drama, Miniseries or Television Movie Haya O Lo Haya', 'Online Film & Television Association 2020 Nominado OFTA Television Award Best Guest Actress in a Drama Series Killing Eve']), ('Dvir Benedek', '1969', ['Reparto', 'Guion', 'Banda sonora'], ['Barash', 'A Matter of Size', 'Wounded Land'], ['Haifa International Film Festival 2015 Ganador Danny Lerner Award Best Actor Barash Jury statement: "An energetic and memorable performance."', 'Awards of the Israeli Film Academy 2015 Nominado Award of the Israeli Film Academy Best Supporting Actor Wounded Land 2009 Nominado Award of the Israeli Film Academy Best Supporting Actor A Matter of Size', 'Awards of the Israeli Television Academy 2013 Nominado Award of the Israeli Television Academy Best Actor in a Comedy Series Ha-Misrad', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Uri Gavriel', '1955', ['Reparto', 'Reparto'], ['El caballero oscuro: La leyenda renace', 'Byzantium', 'Shavua ve Yom'], ["Awards of the Israeli Film Academy 2012 Nominado Award of the Israeli Film Academy Best Actor Balada le'aviv ha'bohe 2011 Nominado Award of the Israeli Film Academy Best Supporting Actor HaFantazia HaGdola shel Simiko HaKatan 2005 Ganador Award of the Israeli Film Academy Best Actor Eize Makom Nifla", 'Karlovy Vary International Film Festival 2005 Ganador Best Actor Eize Makom Nifla Tied with Luca Zingaretti for Alla luce del sole (2005).', 'Oldenburg Film Festival 2016 Ganador Seymour Cassel Award Outstanding Performance by an Ensemble Shavua ve Yom Compartido con: Shai Avivi · Sharon Alexander · Evgenia Dodina · Carmit Mesilati Kaplan · Tomer Capone', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Ruba Salameh', 'NULL', ['Arte', 'Reparto', 'Dirección de arte'], ['su papel en Bar Bahar: Entre dos mundos', 'Between Heaven and Earth', 'Asuntos de familia'], 'NULL'), ('Karim Saleh', '1978', ['Reparto', 'Dirección', 'Dirección de fotografía y cámara'], ['Transparent', 'Counterpart', 'Múnich'], 'NULL'), ('Nisrin Siksik', 'NULL', ['Reparto', 'Reparto'], ['su papel en El atentado', 'Ajami', 'Shkufim'], 'NULL'), ('Bassem Lulu', 'NULL', ['Reparto', 'Reparto'], ['su papel en El atentado'], 'NULL'), ('Ezra Dagan', 'NULL', ['Reparto', 'Reparto'], ['su papel en La lista de Schindler', "Ha'Mishlahat", 'America 3000'], 'NULL')], [], [('Emma Pasarow', '1996', ['Reparto', 'Reparto'], ['Déjate llevar', 'Kickstart My Heart', 'Touch'], 'NULL'), ('Belmont Cameli', '1998', ['Reparto', 'Reparto'], ['Déjate llevar', 'Saved by the Bell', 'Most Guys Are Losers'], 'NULL'), ('Laura Kariuki', '1998', ['Reparto', 'Reparto'], ['Black Lightning', 'Déjate llevar', 'A Complete Unknown'], 'NULL'), ('Kate Bosworth', '1983', ['Reparto', 'Producción', 'Reparto'], ['Superman Returns: El regreso', '21: Black Jack', 'En el filo de las olas'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2007 Nominado Saturn Award Best Actress Superman Returns: El regreso', 'MTV Movie + TV Awards 2003 Nominado MTV Movie Award Best On-Screen Team En el filo de las olas Compartido con: Michelle Rodriguez · Sanoe Lake 2003 Nominado MTV Movie Award Breakthrough Female Performance En el filo de las olas', 'Razzie Awards 2007 Nominado Razzie Award Worst Supporting Actress Superman Returns: El regreso', 'ShoWest Convention, USA 2008 Ganador Special Award Best Ensemble 21: Black Jack Compartido con: Jim Sturgess · Kevin Spacey · Laurence Fishburne · Aaron Yoo · Liza Lapira · Jacob Pitts · Josh Gad', 'Teen Choice Awards 2008 Nominado Teen Choice Award Choice Movie Actress: Drama 21: Black Jack 2006 Nominado Teen Choice Award Movies - Choice Chemistry Superman Returns: El regreso Compartido con: Brandon Routh 2004 Nominado Teen Choice Award Choice Movie Actress - Comedy El chico de tu vida 2004 Nominado Teen Choice Award Choice Movie Blush El chico de tu vida 2004 Nominado Teen Choice Award Choice Movie Liplock El chico de tu vida Compartido con: Topher Grace 1 más'])], [('Joel Ström', '1993', ['Reparto', 'Dirección', 'Guion'], ['Locked-in Syndrome', 'The Simplicity of Life', 'Hühnerwahnsinn - Das Eiskalte Geschäft mit Geflügel'], 'NULL'), ('Sandra Tengvall', 'NULL', ['Producción', 'Reparto', 'Sonido'], ['su papel en The Unthinkable', 'Rememberance', 'Turnpike'], 'NULL'), ('Ida Karolin Johansson', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Jag vill inte', 'Rösterna i mitt huvud', 'The Constant Hunt'], 'NULL'), ('Robin Flemgård', '1985', ['Sonido', 'Dirección', 'Guion'], ['Per Vigilium', 'Paracusia', 'Kärleksbitter'], 'NULL'), ('Jessica Gomez Morales', 'NULL', ['Cámara y electricidad', 'Dirección de fotografía y cámara', 'Edición'], ['su papel en Protea', 'Dive Under', 'Per Vigilium'], 'NULL'), ('Fredrik Pihlgren', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sekten', "It'll Be Different", 'Protea'], 'NULL')], [('Noah Levi', '2001', ['Reparto', 'Banda sonora', 'Reparto'], ['Le coeur en braille', 'Rabbit School', 'Rabbit Academy: El gran robo de los huevos de Pascua'], 'NULL'), ('Friedrich von Thun', '1942', ['Reparto', 'Dirección', 'Guion'], ['La lista de Schindler', 'Die Verbrechen des Professor Capellari', 'Die Hölle'], ['Bambi Awards 1999 Ganador Bambi TV - National Liebe und weitere Katastrophen Compartido con: Senta Berger · Bobby Brederlow', 'Bavarian TV Awards 2007 Ganador Bavarian TV Award Best Actor in a Movie Made for Television (Bester Schauspieler - Fernsehspiel/Fernsehfilm) Helen, Fred und Ted', 'German Television Awards 2007 Nominado German Television Award Best Actor (Bester Schauspieler) Der falsche Tod', 'Golden Cable, Germany 1997 Ganador Audience Award in Gold Favorite TV Doctor For the "Dr. Schwarz und Dr. Martin" series of films. Compartido con: Senta Berger', 'Romy Gala, Austria 2017 Nominado Romy Favorite Actor (Beliebtester Schauspieler) Die Hölle 2006 Nominado Romy Favorite Actor (Beliebtester Schauspieler) Tod eines Keilers']), ('Senta Berger', '1941', ['Reparto', 'Producción', 'Banda sonora'], ['Unter Verdacht', 'Kir Royal', 'Emboscada a Matt Helm'], ['Grimme Award 2010 Ganador Adolf Grimme Award Fiction Frau Böhm sagt Nein Compartido con: Dorothee Schön · Connie Walter · Lavinia Wilson 2003 Ganador Adolf Grimme Award Fiction/Entertainment Unter Verdacht For episode "Verdecktes Spiel (#1.1)". Compartido con: Alexander Adolph · Friedemann Fromm · Rudolf Krause · Gerd Anthoff', 'Baden-Baden TV Film Festival 2011 Ganador Hans Abich Award 2009 Ganador Special Award Best Performance (Beste darstellerische Leistung) Frau Böhm sagt Nein Compartido con: Lavinia Wilson', 'Bambi Awards 2023 Ganador Honorary Bambi Lifetime Achievement 1999 Ganador Bambi TV - National Liebe und weitere Katastrophen Compartido con: Bobby Brederlow · Friedrich von Thun 1990 Ganador Bambi Unknockable Stars 1968 Ganador Bambi Best Actress - National Diabólicamente tuyo', 'Bavarian TV Awards 2016 Ganador Honorary Award of the Bavarian Minister President 2010 Ganador Bavarian TV Award Best Actress in a Movie Made for Television (Beste Schauspielerin - Fernsehspiel/Fernsehfilm) Frau Böhm sagt Nein', 'German Film Awards 2021 Ganador Lifetime Achievement Award']), ('Elise Eikermann', 'NULL', ['Equipos adicionales', 'Reparto', 'Equipos adicionales'], ['su papel en Gatas miedicas', 'Rabbit Academy: El gran robo de los huevos de Pascua'], 'NULL'), ('Liza Simmerlein', 'NULL', ['Reparto', 'Dirección de producción', 'Reparto'], ['su papel en Yakari', 'Los Olchis', 'Rabbit Academy: El gran robo de los huevos de Pascua'], 'NULL'), ('Katharina Straßer', '1984', ['Reparto', 'Banda sonora', 'Reparto'], ['Intuición criminal', 'Wie man leben soll', 'Griechenland'], 'NULL'), ('Sebastian Fitzner', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Violetta', 'Los irregulares', 'All About the Washingtons'], 'NULL'), ('Jule Böwe', '1969', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Gefangene', 'Das kalte Herz', 'Silencio de hielo'], 'NULL'), ('Jannik Endemann', 'NULL', ['Reparto', 'Reparto'], ['su papel en Petronella Apfelmus - Serie', "Leisure Suit Larry: Wet Dreams Don't Dry", 'Harry Potter y la piedra filosofal'], 'NULL'), ('Monty Arnold', '1967', ['Reparto', 'Guion', 'Dirección'], ['Eine Nacht in Wilhelmsburg', 'El hombre deseado', 'Der goldene Nazivampir von Absam 2 - Das Geheimnis von Schloß Kottlitz'], 'NULL')], [('Devanny Pinn', '1989', ['Reparto', 'Producción', 'Reparto'], 'NULL', ['Action on Film International Film Festival, USA 2017 Nominado Action on Film Award Action Film of the Year - Feature California Dreaming Compartido con: Brandon Slagle 2015 Ganador Action on Film Award Best Drama Feature Honrable Mention House of Manson Compartido con: Brandon Slagle · Britt Griffith 2015 Nominado Action on Film Award Best Actress House of Manson', 'Canada International Film Festival 2015 Ganador Rising Star Award Foreign Film Competition House of Manson Compartido con: Brandon Slagle · Britt Griffith', 'PollyGrind Underground Film Festival of Las Vegas 2012 Ganador Best of Categories Best Scream Queen The Black Dahlia Haunting', 'Shockfest Film Festival, US 2021 Nominado Best of Fest Award Best Actress Numbers 2019 Ganador Best of Fest Award Best Actress The Dawn 2019 Nominado Best of Fest Award Best Feature The Dawn Compartido con: Ryan Kiser 2019 Ganador Festival Award Audience Award: Best Feature The Dawn Compartido con: Brandon Slagle · Ryan Kiser 2012 Ganador Best of Fest Award Best Actress The Black Dahlia Haunting', 'California Film Awards 2015 Ganador Silver Award Feature Film Competition House of Manson Compartido con: Brandon Slagle · Britt Griffith']), ('Lynn Lowry', '1947', ['Reparto', 'Producción', 'Guion'], ['Vinieron de dentro de'], ['The Indie Horror Film Festival, Chicago, US 2016 Ganador Best Actress Best Actress The Peripheral', 'Macabre Faire Film Festival 2016 Ganador Macabreite Award Best Actress Feature Model Hunger', 'Buffalo Dreams Fantastic Film Festival 2023 Nominado Festival Prize Best Actress Fang', 'Crimson Screen Horror Film Fest 2015 Nominado Crimmy Best Actress Model Hunger', 'FilmQuest 2015 Nominado FilmQuest Cthulhu Best Actress - Short The Peripheral']), ('Bill Oberst Jr.', '1965', ['Reparto', 'Producción', 'Guion'], ['3 del infierno', 'Mentes criminales', 'Scream Queens'], ['B-Movie Film Festival 2012 Ganador Golden Cob Award Best Rising B Movie Actor A Haunting in Salem', 'Melbourne Underground Film Festival 2016 Ganador Best Actor Best Actor International Film Stressed to Kill', 'New Orleans Film Festival 2013 Ganador Audience Award Best Narrative Feature The Retrieval Joint winner with "King of Herrings" (dirs. Eddie Jemison and Sean Richardson) Compartido con: Chris Eska · Jacob Esquivel · Jason Wehling · Yasu Tanida · Matthew Wiedemann · Ashton Sanders · · · · ·', 'Phoenix Film Festival 2013 Ganador Audience Award Competition Feature Film Awards The Retrieval Compartido con: Chris Eska · Jacob Esquivel · Jason Wehling · Yasu Tanida · Matthew Wiedemann · Ashton Sanders · · · · · · 2013 Ganador Copper Wing Award Best Ensemble Acting The Retrieval Compartido con: Ashton Sanders · Tishuan Scott · Keston John · Christine Horn · Alfonso Freeman · Raven Nicole LeDeatte ·', 'Chicago Horror Film Festival 2024 Nominado Festival Award Best Actor A Stranger in the Woods 2024 Nominado Festival Award Best Psychopath A Stranger in the Woods 2015 Ganador Festival Award Best Actor Circus of the Dead']), ('Jessica Cameron', 'NULL', ['Reparto', 'Producción', 'Dirección'], 'NULL', ['Buenos Aires International Festival of Independent Cinema 2013 Nominado Special Mention Best Picture Truth or Dare Compartido con: Mem Ferda · Jonathan Scott Higgins', 'Chicago Horror Film Festival 2014 Nominado Festival Award Best Feature Truth or Dare Compartido con: Jonathan Scott Higgins · Mem Ferda 2014 Nominado Festival Award Best Director Truth or Dare 2014 Nominado Festival Award Best Actress Truth or Dare', 'PollyGrind Underground Film Festival of Las Vegas 2013 Ganador Audience Choice Favorite Feature Truth or Dare 2013 Ganador Directing Awards Best Female Director Truth or Dare Compartido con: Mem Ferda · Jonathan Scott Higgins 2013 Ganador Pollygrind Royalty Queen Of Pollygrind 2012 Ganador Best of Categories Best Cameo The Black Dahlia Haunting', 'Arizona Underground Film Festival, US 2015 Ganador Festival Prize Best Horror Feature Mania Compartido con: Mem Ferda 2013 Ganador Festival Prize Best Horror Feature Truth or Dare Compartido con: Jonathan Scott Higgins · Mem Ferda', 'The Indie Horror Film Festival, Chicago, US 2014 Ganador Best Feature Truth or Dare Compartido con: Jonathan Scott Higgins · Mem Ferda']), ('Richard Lounello', 'NULL', ['Reparto', 'Guion', 'Dirección'], 'NULL', ['Palm Springs International ShortFest 2023 Ganador Best U.S. Short Best Sci-Fi LifeQuest', 'Bridge Fest, Vancouver 2023 Ganador April Award Best Sci-Fi Short LifeQuest 2023 Ganador April Award Best Director LifeQuest 2023 Ganador Annual Award Best Sci-Fi Short LifeQuest', 'Northeast Film Festival, US 2023 Nominado Festival Award Best Short LifeQuest', 'Buffalo Dreams Fantastic Film Festival 2023 Ganador Jury Prize Outstanding Science Fiction Film LifeQuest', 'IndieFEST Film Awards 2023 Ganador Best of Show Best Short Film LifeQuest 2023 Ganador Best of Show Television Pilot Program LifeQuest']), ('Jennie Russo', 'NULL', ['Reparto', 'Reparto'], ['su papel en She Kills', 'Fang', 'The Horrific Evil Monsters'], 'NULL'), ('Melantha Blackthorne', 'NULL', ['Reparto', 'Especialistas', 'Producción'], ["su papel en Countess Bathoria's Graveyard Picture Show", 'A Grim Becoming', 'Krampus: The Devil Returns'], 'NULL'), ('Valerie Hanna', 'NULL', ['Reparto', 'Producción', 'Guion'], ['Chicago P'], 'NULL'), ('Britt Griffith', 'NULL', ['Cámara y electricidad', 'Reparto', 'Producción'], ['su papel en House of Manson', 'Dead Sea', 'VIViD'], ['Action on Film International Film Festival, USA 2015 Ganador Action on Film Award Best Drama Feature Honrable Mention House of Manson Compartido con: Brandon Slagle · Devanny Pinn', 'Canada International Film Festival 2015 Ganador Rising Star Award Foreign Film Competition House of Manson Compartido con: Brandon Slagle · Devanny Pinn', 'California Film Awards 2015 Ganador Silver Award Feature Film Competition House of Manson Compartido con: Brandon Slagle · Devanny Pinn', 'ECU European Independent Film Festival 2015 Nominado Festival Award Best Non-European Dramatic Feature House of Manson Compartido con: Brandon Slagle · Devanny Pinn', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Bob Bozek', 'NULL', ['Reparto', 'Guion', 'Especialistas'], ['su papel en Emelie', 'Friends to the End', 'Marshall'], 'NULL')], [('Abhinivesh', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en REX, Monster 2 the Final Draft y Scorpion'], 'NULL'), ('Nadiin', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Killer - The Hunt', 'Killer 2 y Red Velvet'], 'NULL'), ('Surendran.A.P', 'NULL', ['Reparto', 'Reparto'], ['su papel en Killer - The Hunt', 'REX y Killer 2'], 'NULL'), ('Ragav', 'NULL', ['Reparto', 'Reparto'], ['su papel en Killer - The Hunt', 'Killer 2 y Demon'], 'NULL'), ('Vijayhari', 'NULL', ['Reparto', 'Reparto'], ['su papel en Killer - The Hunt', 'Killer 2 y REX'], 'NULL')], [('Errol Sack', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Los cazarrecompensas', 'Animal Kingdom', 'Grimm'], 'NULL'), ('Noël Baker', '1988', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Station 4', 'Beyond the Facade', 'Un lugar donde rezar'], 'NULL'), ('George Ortuzar', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Station 4', 'Tropical Tease', 'Cadenas de oro'], 'NULL'), ('Caitlin Muelder', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en ¡Ave, César!', 'History of the World'], 'NULL'), ('Tim May', '1962', ['Transporte', 'Reparto', 'Cámara y electricidad'], ['Iron Man', 'Iron Man 2', 'Disturbia'], 'NULL'), ('George A. Sack Jr.', '1943', ['Especialistas', 'Reparto', 'Producción'], ['Terminator 3: La rebelión de las máquinas', '2 Fast 2 Furious: A todo gas 2', 'Frequency'], 'NULL'), ('Jeff Renfro', 'NULL', ['Reparto', 'Reparto'], ['su papel en Station 4', 'Rusty Tulloch', 'Die for a Dollar'], 'NULL'), ('George D. Cummings', 'NULL', ['Reparto', 'Reparto'], ['su papel en Station 4', 'Rush', 'Station 4'], 'NULL'), ("David 'Shark' Fralick", '1962', ['Reparto', 'Especialistas', 'Producción'], ["Van Damme's Inferno", 'El único', '60 segundos'], 'NULL'), ('Mike McCarty', 'NULL', ['Maquillaje', 'Efectos especiales', 'Equipos adicionales'], ['su papel en Soy el número cuatro', 'Déjà Vu (Cambiando el pasado', 'El sicario de Dios'], 'NULL')], [('Brennan Elliott', '1975', ['Reparto', 'Producción', 'Banda sonora'], ['UnREAL', 'Noche en el museo: El secreto del faraón', 'La maldición de Chucky'], 'NULL'), ('Michael Bailey Smith', '1957', ['Reparto', 'Especialistas', 'Reparto'], ['Las colinas tienen ojos', 'Men in Black II (Hombres de negro II', 'Pesadilla en Elm Street 5: El niño de los sueños'], ['Fright-Fest 2003 Ganador Fright-Fest Award Best Actor in a Comedy Horror Short Blood Shot 2003 Ganador Fright-Fest Award Best Actor in a Short Film Blood Shot', 'Saints and Sinners Film Festival 2003 Ganador Sinners Award Best Actor Blood Shot', 'Fangoria Chainsaw Awards 2006 Ganador Chainsaw Award Bloodiest Beatdown Las colinas tienen ojos For "Michael Bailey Smith vs. Aaron Stanford" Compartido con: Aaron Stanford', 'Maverick Movie Awards 2009 Nominado Maverick Movie Award Best Stunts: Short Thirsty Compartido con: Joe Lynch', 'FANtastic Horror Film Festival, San Diego 2017 Nominado FANtastic Award Best Supporting Actor in a Short Film Girl #2']), ('Brad Dourif', '1950', ['Reparto', 'Reparto'], ['El señor de los anillos: Las dos torres', 'Alguien voló sobre el nido del cuco', 'Arde Mississippi'], ['Academy Awards, USA 1976 Nominado Oscar Best Actor in a Supporting Role Alguien voló sobre el nido del cuco', 'Academy of Science Fiction, Fantasy & Horror Films, USA 1991 Nominado Saturn Award Best Supporting Actor El exorcista III', 'BAFTA Awards 1977 Ganador BAFTA Film Award Best Supporting Actor Alguien voló sobre el nido del cuco', 'Chicago Film Critics Association Awards 1989 Nominado CFCA Award Best Supporting Actor Arde Mississippi', 'Primetime Emmy Awards 2004 Nominado Primetime Emmy Outstanding Supporting Actor in a Drama Series Deadwood For playing "Doc Cochran".']), ('Lance Henriksen', '1940', ['Reparto', 'Producción', 'Equipos adicionales'], ['Aliens: El regreso', 'Blanco humano', 'Alien vs'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2009 Ganador Life Career Award 1999 Nominado Saturn Award Best Genre TV Actor Millennium 1997 Nominado Saturn Award Best Genre TV Actor Millennium 1994 Ganador Saturn Award Best Supporting Actor Blanco humano 1988 Nominado Saturn Award Best Actor Pacto de sangre', 'Chicago Film Critics Association Awards 1990 Nominado CFCA Award Best Supporting Actor Johnny el Guapo', 'Fantafestival 1991 Ganador Best Actor El péndulo de la muerte', 'Golden Globes, USA 1999 Nominado Golden Globe Best Performance by an Actor in a Television Series - Drama Millennium 1998 Nominado Golden Globe Best Performance by an Actor in a Television Series - Drama Millennium 1997 Nominado Golden Globe Best Performance by an Actor in a Television Series - Drama Millennium', 'Satellite Awards 1999 Nominado Golden Satellite Award Best Actor in a Supporting Role in a Miniseries or a Motion Picture Made for Television The Day Lincoln Was Shot']), ('Christopher Lambert', '1957', ['Reparto', 'Producción', 'Guion'], ['Los inmortales', 'Mortal Kombat', 'Beowulf, la leyenda'], ['César Awards, France 1986 Ganador César Best Actor (Meilleur acteur) Subway (En busca de Freddy)', 'Monte-Carlo TV Festival 2014 Nominado Golden Nymph Outstanding Actor in a Mini-Series La source', 'Jupiter Award 1987 Nominado Jupiter Award Best International Actor Los inmortales 1985 Nominado Jupiter Award Best International Actor Greystoke: La leyenda de Tarzán, el rey de los monos', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Martin Klebba', '1969', ['Reparto', 'Especialistas', 'Equipos adicionales'], ['The Electric State', 'Piratas del Caribe: La maldición de la Perla Negra', 'Knee High P'], 'NULL'), ('Assaf Cohen', 'NULL', ['Reparto', 'Reparto'], ['Entourage (El séquito', 'Elemental', 'This Is Us'], 'NULL'), ('Azita Ghanizada', 'NULL', ['Reparto', 'Producción', 'Banda sonora'], ['La perfecta desconocida', 'Alphas', 'United States of Al'], 'NULL'), ('Joey Naber', 'NULL', ['Reparto', 'Equipos adicionales', 'Sonido'], ['su papel en Alita: Ángel de combate', 'X-Men: Días del futuro pasado', 'Five Wishes'], 'NULL'), ('Herzl Tobey', '1973', ['Reparto', 'Equipos adicionales', 'Guion'], ['Zohan: Licencia para peinar', 'Shnot HaShmonim', 'The Shield: Al margen de la ley'], ['Miami International Science Fiction Film Festival 2021 Nominado Short Film Best Short Spaceship Compartido con: Gal Ziv', 'Atlanta Sci-Fi Film Festival 2021 Nominado Jury Award Best Sci-fi Short Spaceship Compartido con: Gal Ziv', 'Green Fest 2021 Nominado Jury Prize for Author Best Short Film Spaceship Compartido con: Gal Ziv', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona'])], [('Sakshi Aandhale', 'NULL', ['Reparto', 'Reparto'], ['su papel en Koytaa Ek Sangharsh', 'Pirem'], 'NULL')], [('Olivier McGraw', 'NULL', ['Reparto', 'Reparto'], ['su papel en Come What May'], 'NULL'), ('Arthur Wilde', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Rogue One: Una historia de Star Wars', 'Lack of Evidence', 'Control'], 'NULL')], [('Elaina Adrianna', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['Solve', 'Silence', 'Millennial Dilemma'], 'NULL'), ('Jamin Keene', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Vinegar Water: Virtual Staged Reading', 'Every Screenwriters Group Meeting Ever', 'Linda Evans Is My Spirit Animal'], 'NULL'), ('Heather Mattheisen', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Gelato', 'Dollhouse', 'Cody Allen: Black & Blue'], 'NULL'), ('Andrew Levi Hall', 'NULL', ['Reparto', 'Edición', 'Dirección de fotografía y cámara'], ['su papel en See Me', 'The Other Seven', '400 to Oahu'], 'NULL')], [('Woody Fu', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en The Other Two', 'Awkwafina es Nora de Queens', 'El espíritu de la Navidad'], 'NULL'), ('Adelind Horan', '1988', ['Reparto', 'Guion', 'Producción'], ['The Peripheral', 'The Deuce (Las crónicas de Times Square', 'The Punisher'], 'NULL'), ('Tony Macht', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Thynspo', 'Bad Dream', 'Listen'], 'NULL'), ('April Matthis', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en New Amsterdam', 'Instinct', 'BlackCard'], 'NULL'), ('Derek Smith', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bad Dream'], 'NULL')], [], [('Radomir Jordanovic', 'NULL', ['Dirección', 'Dirección de fotografía y cámara', 'Guion'], ['su papel en Pay the Lady', "Christine's Wager y Lethologica"], ['Brightside Tavern Film Festival 2024 Nominado Jury Prize Best Cinematography Lethologica 2024 Nominado Jury Prize Best Concept Lethologica', "Avalonia Festival 2023 Ganador Jury Prize Best Comedy 3 Minutes & Under Carl Goes to the Hospital 2021 Ganador Jury Prize One Actor or Actress Movie or Monologue 3 Minutes & Under Absolute Soup 2019 Ganador Jury Prize Best Comedy 3 Minutes & Under Kevin's Thermodynamics Lesson", 'MLC Awards 2023 Ganador Best Voiceover Recording Quality Absolute Soup', 'Golden Wheat Awards 2021 Ganador Golden Wheat Award Best Short Film Tim Bradley', 'Golden Harvest Film Festival 2021 Ganador Best Short Best Short A Matter of Two Minutes'])], [('Brandon Alan Smith', 'NULL', ['Reparto', 'Especialistas', 'Producción'], ['su papel en Gotham', 'Brawl in Cell Block 99', 'The Amityville Harvest'], 'NULL'), ('Telvin Griffin', 'NULL', ['Reparto', 'Reparto'], ['Babylon', 'Deputy', 'Black-ish'], 'NULL'), ('Shelly Snellman', 'NULL', ['Producción', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en A Blue Shroud', 'Parkour', 'TV for Monsters'], ['IndieFEST Film Awards 2021 Ganador Award of Merit Cinematography A Blue Shroud Compartido con: C. Clifford Jones · Jason Snellman 2021 Ganador Award of Merit Best Social Justice/Liberation/Protest Film A Blue Shroud Compartido con: Jason Snellman', 'Mindfield Film Festival Albuquerque 2021 Ganador Gold Award Best Experimental Short Parkour Compartido con: Jason Snellman · Emma Servant · Kiersten Thomsen · Adam Daniel · Liz Izzo · Moises Ignacio Garcia · ·', 'Los Angeles Cinematography AWARDS (LACA) 2021 Ganador March Award Best American Film A Blue Shroud Compartido con: Jason Snellman', 'Beyond the Curve International Film Festival 2021 Nominado April Award Best Short Film Parkour Compartido con: Jason Snellman · Kiersten Thomsen · Emma Servant · Adam Daniel · Liz Izzo · Jeremey Adam Rey', 'World Film Festival in Cannes 2021 Nominado Monthly Award Best Super Short Film Parkour Compartido con: Jason Snellman · Kiersten Thomsen · Emma Servant · Adam Daniel · Liz Izzo · Jeremey Adam Rey']), ('Alexandra Siegel', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bosch', 'De culo y cuesta abajo', 'Status Update'], 'NULL'), ('Nathan Marlow', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nash Bridges', 'Hood Damnation y Tupilak'], 'NULL'), ('Chad Addison', 'NULL', ['Reparto', 'Guion', 'Dirección de la segunda unidad o asistencia a dirección'], ['Más propenso a morir', 'NCIS: Nueva Orleans', 'All Rise'], 'NULL'), ('Leif Gantvoort', '1972', ['Reparto', 'Dirección', 'Guion'], ['Navy: Investigación criminal', 'The Amazing Spider-Man', 'Superpoli en Las Vegas'], ['Method Fest 2009 Nominado Best Actor Feature Film Lunatics, Lovers & Poets', 'Wisconsin Film Festival 2017 Ganador Jury Award Jury Award The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough', 'Independent Filmmakers Showcase IFS Film Festival, US 2017 Ganador IFS Award Best Comedy Award The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough', 'South Dakota Film Festival 2019 Ganador Festival Award Best Comedy Horror Heavy Stuff', 'Phoenix Comicon Film Festival 2017 Ganador Jury Prize Best of Festival The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough']), ("Travis 'T.J.' Johnson", 'NULL', ['Reparto', 'Reparto'], ['su papel en Love XO', 'Night Out', 'A Blue Shroud'], 'NULL'), ("I'ke Mitchell", 'NULL', ['Reparto', 'Reparto'], ['su papel en A Blue Shroud'], 'NULL'), ('Kourtney Jay', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Just Cause', 'Desperate Morons', "I Still Haven't Found the Droids I'm Looking For"], 'NULL')], [('Adrien Deschamps', 'NULL', ['Reparto', 'Reparto'], ['su papel en Le discours philosophique de Pierre du vendredi soir au samedi apres-midi'], 'NULL'), ('Gabriel Dostes', 'NULL', ['Cámara y electricidad', 'Reparto', 'Guion'], ['su papel en Venganza: Conexión Estambul', 'Jessica Forever', 'Lock-Down'], 'NULL'), ('Noemie Legrand', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Hector and the others', 'Le discours philosophique de Pierre du vendredi soir au samedi apres-midi', 'Ascenseur'], 'NULL'), ('Mia Smith', 'NULL', ['Reparto', 'Reparto'], ['su papel en Le discours philosophique de Pierre du vendredi soir au samedi apres-midi'], 'NULL')], [], [('Mariya Dmitrieva', '1998', ['Reparto', 'Reparto'], ['Ostorozhno - lyubov', "A Killer's Mind", 'Novogodniy shef'], 'NULL'), ('Elijah Khodyrev', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Choice', 'A Bedtime Story'], 'NULL')], [('Josh Salatin', 'NULL', ['Reparto', 'Reparto'], ['su papel en Amor a segunda vista', 'Zoo', 'The Following'], 'NULL'), ('Hale Lytle', '2001', ['Reparto', 'Reparto'], ["Don't Come Back from the Moon", 'Killer', 'Young Bodies Heal Quickly'], 'NULL'), ('Lindsay Burdge', '1984', ['Reparto', 'Dirección de casting', 'Guion'], ['Thirst Street', 'The Midnight Swim', 'La invitación'], ['Oldenburg Film Festival 2017 Ganador Seymour Cassel Award Outstanding Performance by an Actress Thirst Street', 'Sun Valley Film Festival, US 2015 Ganador One In A Million Award Narrative The Midnight Swim Compartido con: Sarah Adina Smith · Jennifer Lafleur · Aleksa Palladino · Ross Partridge', 'Starz Denver Film Festival 2014 Ganador American Independent Award for Best Film Best Film The Midnight Swim Compartido con: Sarah Adina Smith · Jennifer Lafleur · Aleksa Palladino · Ross Partridge', 'Riviera International Film Festival 2018 Nominado Jury Prize Best Actress Thirst Street', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Michael Kulukundis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Fort Apache'], 'NULL'), ('Fayelyn Bilodeau', 'NULL', ['Reparto', 'Reparto'], ['su papel en El mentalista', 'Bajo su dominio', 'The Invisible Mother'], 'NULL'), ('Sally Sockwell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Doris & The Intern', 'Alamo Bay (La bahía del odio', 'Una mujer de negocios'], 'NULL'), ('Connor Hurley', 'NULL', ['Edición', 'Producción', 'Dirección'], ['su papel en The Naturalist', 'Skook', 'The Other Man'], ['Brooklyn Film Festival 2013 Ganador Festival Award Spirit Award for Short Narrative The Naturalist', 'New Orleans Film Festival 2013 Ganador Grand Jury Award Narrative Feature Skook Compartido con: Ashley Pishock 2013 Nominado Narrative Short Award Best Short Film The Naturalist', 'Puchon International Fantastic Film Festival 2013 Nominado Grand Prize for Short Film The Naturalist', 'Indianapolis International Film Festival 2014 Nominado American Spectrum Prize Best Narrative Feature Skook', 'Reykjavik International Film Festival 2014 Nominado RIFF Golden Egg Award The Naturalist']), ('Dan Hicks', '1951', ['Reparto', 'Producción', 'Equipos adicionales'], ['Darkman', 'Terroríficamente muertos', 'Intruso en la noche'], 'NULL')], [('Toni Luoma-Nirva', 'NULL', ['Edición', 'Edición', 'Reparto'], ['su papel en Fight or Flight', 'The Pain of Creation', 'Toni Air'], 'NULL'), ('Anton Jokikunnas', 'NULL', ['Cámara y electricidad', 'Dirección', 'Producción'], ['su papel en Angels of the Dark MC, Layehaye Doroogh', 'Viraali'], 'NULL'), ('Juha Mäkinen', 'NULL', ['Reparto', 'Edición', 'Dirección'], ['su papel en Live'], 'NULL')], [('Judy Krause', 'NULL', ['Reparto', 'Producción', 'Dirección de fotografía y cámara'], ['su papel en Brooklyn in July', 'A Rogue in Londinium', "Adams' Apples"], 'NULL'), ('Travis Mitchell', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Spinning Man', 'Partitioned Heart', 'Nevertheless, She Auditioned'], 'NULL'), ('Bob Rutan', 'NULL', ['Equipos adicionales', 'Reparto', 'Producción'], ['su papel en Beached', 'Ten Dollars', "Macy's Fourth of July Fireworks Spectacular"], 'NULL')], [('Tovino Thomas', '1989', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], ['Rayo Murali', '2018', 'Kala'], ['Asianet Film Awards 2019 Ganador Asianet Film Award Performer of the Year 2018 Ganador Asianet Film Award Performer of the Year Mayaanadhi', 'Filmfare Awards South 2022 Nominado Filmfare Award - Malayalam Film Industry Best Actor Rayo Murali 2019 Nominado Filmfare Award - Malayalam Film Industry Best Actor Theevandi 2018 Nominado Filmfare Award - Malayalam Film Industry Best Actor Mayaanadhi', 'South Indian International Movie Awards 2023 Ganador SIIMA - Malayalam Best Actor in a Leading Role Thallumaala 2022 Ganador SIIMA - Malayalam Best Actor in a Leading Role Rayo MuraliKala 2021 Nominado SIIMA - Malayalam Best Actor in a Leading Role Forensic 2019 Ganador SIIMA - Malayalam Best Actor in a Leading Role Theevandi', 'Asiavision Awards 2015 Ganador Asiavision Film Award Best Actor - Supporting Role Ennu Ninte Moideen', 'Vanitha Film Awards 2019 Ganador Vanitha Film Award Most Popular Actor Oru Kuprasidha PayyanEnte Ummante Peru 2018 Ganador Vanitha Film Award Best Romantic Hero Mayaanadhi']), ('Samyuktha Menon', '1995', ['Reparto', 'Reparto'], ['Bheemla Nayak', 'Devil', 'Ram'], 'NULL'), ('Shalu Rahim', 'NULL', ['Reparto', 'Reparto'], ['su papel en Maradona', 'Edakkad Battalion 06', 'Ottakkoru Kaamukan'], 'NULL'), ('Karthik Vishnu', 'NULL', ['Reparto', 'Reparto'], ['su papel en Kidu', 'Kali', 'Edakkad Battalion 06'], 'NULL'), ('Nirmal Palazhi', 'NULL', ['Reparto', 'Música', 'Banda sonora'], ['su papel en Big Brother', 'Angane Njanum Premichu', 'Scene Number 36 Malavika Veedu'], 'NULL'), ('Dheeraj Denny', 'NULL', ['Reparto', 'Reparto'], ['su papel en Kalki', 'Karnan Napoleon Bhagat Singh', 'Edakkad Battalion 06'], 'NULL'), ('Sankar Induchoodan', '1996', ['Reparto', 'Reparto'], ['Hridayam', 'Edakkad Battalion 06', 'Rakshadhikari Baiju Oppu'], 'NULL'), ('Malavika Menon', '1998', ['Reparto', 'Reparto'], ['Hero', 'Kaduva', 'Edakkad Battalion 06'], 'NULL'), ('P. Balachandran', '1952', ['Reparto', 'Guion', 'Dirección'], ['Ivan Megharoopan', 'Kammatti Paadam', 'Rayo Murali'], 'NULL'), ('Rekha', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dasharatham', 'In Harihar Nagar', 'Dasavatharam'], 'NULL')]]}


In [3]:
df_todo=pd.DataFrame(dic_peli)
df_todo
#convierto en cvs 
df_todo.to_csv('actores_pelis_500_600.csv', index=False)